In [1]:
# import accelerate

In [2]:
%env HF_ENDPOINT=https://hf-mirror.com
import os
os.environ['HF_HOME'] = '/data1/ckw'
os.environ['HF_ENDPOINT']='https://hf-mirror.com'

env: HF_ENDPOINT=https://hf-mirror.com


首先，我们构造Tokenizer

In [3]:
import regex as re
import base64
import os
import json
import tiktoken
from torch import TensorType
from typing import List, Optional, Union, Dict, Any
from transformers import PreTrainedTokenizer
from transformers.utils import logging, PaddingStrategy
from transformers.tokenization_utils_base import EncodedInput, BatchEncoding

`ChatGLM4Tokenizer` 类是一个自定义的 `PreTrainedTokenizer` 类，用于处理特定的 GLM-4 模型的分词需求。这个类主要有以下结构和成员函数：

### 类结构
- **属性**：
  - `vocab_files_names`：包含词汇文件名的字典。
  - `model_input_names`：包含模型输入名的列表。
  - `vocab_file`：词汇文件路径。
  - `name`：分词器的名称。
  - `pat_str`：正则表达式字符串，用于分词。
  - `encode_special_tokens`：是否编码特殊字符。
  - `mergeable_ranks`：可合并的词汇排名。
  - `tokenizer`：基于 `tiktoken` 库的编码器。
  - `decoder`：解码器，映射词汇排名到词汇。
  - `n_words`：词汇表大小。

### 成员函数

- **初始化函数**：
  ```python
  def __init__(self, vocab_file, padding_side="left", clean_up_tokenization_spaces=False, encode_special_tokens=False, **kwargs)
  ```
  初始化 `ChatGLM4Tokenizer` 类，加载词汇文件，设置正则表达式和分词器。

- **词汇表大小属性**：
  ```python
  @property
  def vocab_size(self)
  ```
  返回词汇表大小。

- **获取词汇表**：
  ```python
  def get_vocab(self)
  ```
  返回词汇表字典。

- **将 tokens 转换为字符串**：
  ```python
  def convert_tokens_to_string(self, tokens: List[Union[bytes, str]]) -> str
  ```
  将 tokens 序列转换为字符串。

- **分词**：
  ```python
  def _tokenize(self, text, **kwargs)
  ```
  将文本分词为 token 列表。

- **将 token 转换为 ID**：
  ```python
  def _convert_token_to_id(self, token)
  ```
  将 token（字符串）转换为 ID。

- **将 ID 转换为 token**：
  ```python
  def _convert_id_to_token(self, index)
  ```
  将 ID（整数）转换为 token（字符串）。

- **保存词汇表**：
  ```python
  def save_vocabulary(self, save_directory, filename_prefix=None)
  ```
  保存词汇表到指定目录。

- **获取前缀 tokens**：
  ```python
  def get_prefix_tokens(self)
  ```
  返回前缀 tokens 列表。

- **构建单条消息**：
  ```python
  def build_single_message(self, role, metadata, message, tokenize=True)
  ```
  构建单条消息，包含角色、元数据和消息内容。

- **应用聊天模板**：
  ```python
  def apply_chat_template(self, conversation, add_generation_prompt=False, tokenize=True, padding=False, truncation=False, max_length=None, return_tensors=None, return_dict=False, tokenizer_kwargs=None, add_special_tokens=True, **kwargs)
  ```
  将会话数据应用到聊天模板中。

- **构建带有特殊 token 的输入**：
  ```python
  def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None)
  ```
  构建带有特殊 token 的输入。

- **填充输入**：
  ```python
  def _pad(self, encoded_inputs, max_length=None, padding_strategy=PaddingStrategy.DO_NOT_PAD, pad_to_multiple_of=None, return_attention_mask=None)
  ```
  填充编码后的输入，确保输入长度一致。

### 类中的主要函数和方法概述

- **初始化**：`__init__` 函数用于初始化分词器，包括加载词汇表、正则表达式的编译和编码器的初始化。
- **获取词汇表和大小**：`get_vocab` 和 `vocab_size` 属性用于获取词汇表及其大小。
- **分词及转换**：`_tokenize`、`convert_tokens_to_string`、`_convert_token_to_id` 和 `_convert_id_to_token` 函数用于实现文本的分词及 tokens 和 IDs 之间的转换。
- **保存和加载**：`save_vocabulary` 函数用于保存词汇表到指定目录。
- **会话处理**：`get_prefix_tokens`、`build_single_message` 和 `apply_chat_template` 函数用于处理会话数据和模板应用。
- **输入处理**：`build_inputs_with_special_tokens` 和 `_pad` 函数用于处理模型输入，确保输入格式和长度符合要求。

通过以上结构和成员函数，`ChatGLM4Tokenizer` 类实现了一个完整的分词器功能，能够处理特定的 GLM-4 模型的分词需求。

In [61]:
class ChatGLM4Tokenizer(PreTrainedTokenizer):
    # 定义词汇文件名和模型输入名
    vocab_files_names = {"vocab_file": "tokenizer.model"}
    model_input_names = ["input_ids", "attention_mask", "position_ids"]

    def __init__(
            self,
            vocab_file,
            padding_side="left",
            clean_up_tokenization_spaces=False,
            encode_special_tokens=False,
            **kwargs
    ):
        # 初始化一些基础属性
        self.name = "GLM4Tokenizer"
        self.vocab_file = vocab_file
        
        # 正则表达式模式字符串，用于分词
        pat_str = "(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+"
        # 编译正则表达式
        self.pat_str = regex.compile(pat_str)
        # self.pat_str = re.compile(pat_str)
        # 是否编码特殊字符
        self.encode_special_tokens = encode_special_tokens

        # 用于存储可合并的词汇排名
        mergeable_ranks = {}
        # 读取词汇文件
        with open(vocab_file) as f:
            for line in f:
                token, rank = line.strip().split()  # 读取每一行，获取词汇和其对应的排名
                rank = int(rank)  # 将排名转换为整数
                token = base64.b64decode(token)  # 解码词汇
                mergeable_ranks[token] = rank  # 存储到词汇排名字典中

        self.mergeable_ranks = mergeable_ranks

        # 初始化编码器，使用 tiktoken 库
        self.tokenizer = tiktoken.Encoding(
            name="my_tokenizer",
            pat_str=pat_str,
            mergeable_ranks=mergeable_ranks,
            special_tokens={}
        )
        # 解码器，映射排名到词汇
        self.decoder = {rank: token for token, rank in mergeable_ranks.items()}
        # 词汇表大小
        self.n_words = len(self.decoder)

        # 调用父类的初始化方法
        super().__init__(
            padding_side=padding_side,
            clean_up_tokenization_spaces=clean_up_tokenization_spaces,
            **kwargs
        )

    @property
    def vocab_size(self):
        # 返回词汇表大小
        return self.n_words

    def get_vocab(self):
        """ 返回词汇表字典 """
        vocab = {self._convert_id_to_token(i): i for i in range(self.vocab_size)}
        vocab.update(self.added_tokens_encoder)
        return vocab

    def convert_tokens_to_string(self, tokens: List[Union[bytes, str]]) -> str:
        """
        将 tokens 序列转换为字符串。
        """
        text = ""
        temp = b""
        for t in tokens:
            if isinstance(t, str):
                if temp:
                    text += temp.decode("utf-8", errors="replace")
                    temp = b""
                text += t
            elif isinstance(t, bytes):
                temp += t
            else:
                raise TypeError("token should only be of type bytes or str")
        if temp:
            text += temp.decode("utf-8", errors="replace")
        return text

    def _tokenize(self, text, **kwargs):
        # 使用正则表达式和编码器对文本进行分词
        tokens = []
        ids = self.tokenizer.encode(text)
        for t in ids:
            tokens.append(self.decoder[t])
        return tokens

    def _convert_token_to_id(self, token):
        """ 将 token (字符串) 转换为 id (整数) """
        return self.mergeable_ranks[token]

    def _convert_id_to_token(self, index):
        """ 将 id (整数) 转换为 token (字符串) """
        return self.decoder.get(index, "")

    def save_vocabulary(self, save_directory, filename_prefix=None):
        """
        将词汇表和特殊字符文件保存到指定目录。

        Args:
            save_directory (`str`): 保存词汇表的目录。
            filename_prefix (`str`, *optional*): 保存文件名的前缀。

        Returns:
            `Tuple(str)`: 保存的文件路径。
        """
        if os.path.isdir(save_directory):
            vocab_file = os.path.join(
                save_directory, self.vocab_files_names["vocab_file"]
            )
        else:
            vocab_file = save_directory

        with open(self.vocab_file, 'rb') as fin:
            proto_str = fin.read()

        with open(vocab_file, "wb") as writer:
            writer.write(proto_str)

        return (vocab_file,)

    def get_prefix_tokens(self):
        # 返回前缀 tokens 列表
        prefix_tokens = [self.convert_tokens_to_ids("[gMASK]"), self.convert_tokens_to_ids("<sop>")]
        return prefix_tokens

    def build_single_message(self, role, metadata, message, tokenize=True):
        # 构建单条消息，包含角色、元数据和消息内容
        assert role in ["system", "user", "assistant", "observation"], role
        if tokenize:
            role_tokens = [self.convert_tokens_to_ids(f"<|{role}|>")] + self.tokenizer.encode(f"{metadata}\n",
                                                                                              disallowed_special=())
            message_tokens = self.tokenizer.encode(message, disallowed_special=())
            tokens = role_tokens + message_tokens
            return tokens
        else:
            return str(f"<|{role}|>{metadata}\n{message}")

    def apply_chat_template(
            self,
            conversation: Union[List[Dict[str, str]], List[List[Dict[str, str]]], "Conversation"],
            add_generation_prompt: bool = False,
            tokenize: bool = True,
            padding: bool = False,
            truncation: bool = False,
            max_length: Optional[int] = None,
            return_tensors: Optional[Union[str, TensorType]] = None,
            return_dict: bool = False,
            tokenizer_kwargs: Optional[Dict[str, Any]] = None,
            add_special_tokens: bool = True,
            **kwargs,
    ) -> Union[str, List[int], List[str], List[List[int]], BatchEncoding]:
    
        if return_dict and not tokenize:
            raise ValueError(
                "`return_dict=True` is incompatible with `tokenize=False`, because there is no dict "
                "of tokenizer outputs to return."
            )
    
        def handle_single_conversation(conversation):
            input_ids = self.get_prefix_tokens() if add_special_tokens else []
            input_message = "[gMASK]<sop>" if add_special_tokens else ""
            for item in conversation:
                if item.get("tools"):
                    tools = item["tools"]
                    content = "你是一个名为 GLM-4 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。"
                    for tool in tools:
                        if tool["type"] == "function":
                            function = tool["function"]
                            content += f"\n\n## {function['name']}\n\n{json.dumps(function, ensure_ascii=False, indent=4)}"
                            content += "\n在调用上述函数时，请使用 Json 格式表示调用的参数。"
                        elif tool["type"] == "python":
                            content += "\n\n## python\n\n当你向 `python` 发送包含 Python 代码的消息时，该代码将会在一个有状态的 Jupyter notebook 环境中执行。\n`python` 返回代码执行的输出，或在执行 60 秒后返回超时。\n`/mnt/data` 将会持久化存储你的文件。在此会话中，`python` 无法访问互联网。不要使用 `python` 进行任何网络请求或者在线 API 调用，这些在线内容的访问将不会成功。"
                        elif tool["type"] == "simple_browser":
                            content += "\n\n## simple_browser\n\n你可以使用 `simple_browser` 工具。该工具支持以下函数：\n`search(query: str, recency_days: int)`：使用搜索引擎进行查询并显示结果，可以使用 `recency_days` 参数控制搜索内容的时效性。\n`mclick(ids: list[int])`：获取一系列指定 id 的页面内容。每次调用时，须选择3-10个页面。选择多个角度的页面，同时尽可能选择可信任的信息来源。考虑到部分页面是无法加载的，你也可以多打开一些可能有用的页面而不用担心内容过多。\n`open_url(url: str)`：打开指定的 URL。\n\n使用 `【{引用 id}†{引用文本}】` 来引用内容。\n\n操作步骤：1. 使用 `search` 来获得信息列表; 2. 使用 `mclick` 来获取指定 ID 页面的内容; 3. 根据获得的内容进行回复。在回复中应当引用信息来源。\n 如果用户提供了 URL，也可以用 `open_url` 直接打开页面。\n如果初次搜索结果没有找到合适的信息，也可以再次使用 `search` 进行搜索。"
                        elif tool["type"] == "cogview":
                            content += "\n\n## cogview\n\n如果用户的请求中包含了对图像的描述，你可以使用 `cogview` 来生成图像并展示给用户。你需要向 `cogview` 发送图像描述，规则：\n- 发送给 `cogview` 的消息必须使用英语。用户的中文描述必须完全翻译为英语。\n- 应当尽可能详细地描述图像生成的需求，需求描述约 100 英文单词。\n- 保持用户原始描述的意图。不要虚构内容或者没见过的人物。\n- 如无特殊说明，所在地为中国，持有中国立场并遵循中国社会主义价值观。"
                        else:
                            raise NotImplementedError(f"Unknown tool type {tool['type']}")
                    input = self.build_single_message("system", "", content, tokenize=tokenize)
                    if tokenize:
                        input_ids.extend(input)
                    else:
                        input_message += input
                if item["content"]:
                    input = self.build_single_message(
                        item["role"],
                        item.get("metadata", ""),
                        item["content"],
                        tokenize=tokenize
                    )
                    if tokenize:
                        input_ids.extend(input)
                    else:
                        input_message += input
            if add_generation_prompt:
                if tokenize:
                    input_ids.extend([self.convert_tokens_to_ids("<|assistant|>")])
                else:
                    input_message += "<|assistant|>"
                # if tokenize:
                #     input_ids.extend([self.convert_tokens_to_ids("[gMASK]")])  # 使用特殊标记代替空字符串
                # else:
                #     input_message += "[gMASK]"
    
            return input_ids if tokenize else input_message
    
        # 处理不同会话格式的主逻辑
        if isinstance(conversation, list) and all(isinstance(i, dict) for i in conversation):
            result = handle_single_conversation(conversation)
        elif isinstance(conversation, list) and all(isinstance(i, list) for i in conversation):
            result = [handle_single_conversation(c) for c in conversation]
        elif hasattr(conversation, "messages"):
            result = handle_single_conversation(conversation.messages)
        else:
            raise ValueError("Invalid conversation format")
    
        if tokenize:
            output = self.batch_encode_plus(
                [result] if isinstance(result[0], int) else result,
                padding=padding,
                truncation=truncation,
                max_length=max_length,
                return_tensors=return_tensors,
                is_split_into_words=True,
                add_special_tokens=False
            )
            if return_dict:
                return output
            else:
                return output["input_ids"]
        else:
            return result



    def build_inputs_with_special_tokens(
            self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """
        构建模型输入，适用于序列分类任务，通过连接和添加特殊 tokens。
        BERT 序列格式：
        - 单序列: `[CLS] X [SEP]`
        - 序列对: `[CLS] A [SEP] B [SEP]`

        Args:
            token_ids_0 (`List[int]`): 要添加特殊 tokens 的 ID 列表。
            token_ids_1 (`List[int]`, *optional*): 可选的第二个 ID 列表，表示序列对。

        Returns:
            `List[int]`: 添加了适当特殊 tokens 的输入 ID 列表。
        """
        prefix_tokens = self.get_prefix_tokens()
        token_ids_0 = prefix_tokens + token_ids_0
        if token_ids_1 is not None:
            token_ids_0 = token_ids_0 + token_ids_1 + [self.convert_tokens_to_ids("<eos>")]
        return token_ids_0

    def _pad(
            self,
            encoded_inputs: Union[Dict[str, EncodedInput], BatchEncoding],
            max_length: Optional[int] = None,
            padding_strategy: PaddingStrategy = PaddingStrategy.DO_NOT_PAD,
            pad_to_multiple_of: Optional[int] = None,
            return_attention_mask: Optional[bool] = None,
    ) -> dict:
        """
        填充编码后的输入（左右填充，并预定义长度或批处理中的最大长度）

        Args:
            encoded_inputs: 包含编码后的输入 (`List[int]`) 或批处理的输入 (`List[List[int]]`) 的字典。
            max_length: 返回列表的最大长度，并可选的填充长度。
            padding_strategy: 用于填充的策略。
                - PaddingStrategy.LONGEST: 填充到批处理中的最长序列
                - PaddingStrategy.MAX_LENGTH: 填充到最大长度（默认）
                - PaddingStrategy.DO_NOT_PAD: 不填充
                填充策略由 self.padding_side 定义：
                    - 'left': 在序列左侧填充
                    - 'right': 在序列右侧填充
            pad_to_multiple_of: (可选) 整数，如果设置将填充序列为该值的倍数。
            return_attention_mask: (可选) 设置为 False 以避免返回注意力掩码（默认：根据模型具体情况设置）
        """
        # 确保填充侧为 'left'
        assert self.padding_side == "left"

        required_input = encoded_inputs[self.model_input_names[0]]
        seq_length = len(required_input)

        if padding_strategy == PaddingStrategy.LONGEST:
            max_length = len(required_input)

        if max_length is not None and pad_to_multiple_of is not None and (max_length % pad_to_multiple_of != 0):
            max_length = ((max_length // pad_to_multiple_of) + 1) * pad_to_multiple_of

        needs_to_be_padded = padding_strategy != PaddingStrategy.DO_NOT_PAD and len(required_input) != max_length

        # 如果没有注意力掩码，初始化
        if "attention_mask" not in encoded_inputs:
            encoded_inputs["attention_mask"] = [1] * seq_length

        if "position_ids" not in encoded_inputs:
            encoded_inputs["position_ids"] = list(range(seq_length))

        if needs_to_be_padded:
            difference = max_length - len(required_input)

            if "attention_mask" in encoded_inputs:
                encoded_inputs["attention_mask"] = [0] * difference + encoded_inputs["attention_mask"]
            if "position_ids" in encoded_inputs:
                encoded_inputs["position_ids"] = [0] * difference + encoded_inputs["position_ids"]
            encoded_inputs[self.model_input_names[0]] = [self.pad_token_id] * difference + required_input

        return encoded_inputs


### 关键点解释

1. **`mergeable_ranks`**:
   - 存储可合并词汇的排名信息。读取词汇文件时，每个词汇都有一个对应的排名（整数），这些词汇被解码为原始字符串，然后存储在 `mergeable_ranks` 字典中。

2. **`pat_str`**:
   - 正则表达式字符串，用于定义分词模式。这段正则表达式主要用于匹配常见的英语缩写、单词、数字和其他非字母数字字符。编译后的正则表达式存储在 `self.pat_str` 中，用于分词器。

3. **`self.tokenizer`**:
   - 使用 `tiktoken` 库创建的编码器。通过提供的正则表达式模式和可合并的词汇排名，初始化一个自定义的分词器。

4. **`self.decoder`**:
   - 解码器，映射词汇排名到词汇。使用 `mergeable_ranks` 字典创建的逆向字典，用于从 ID 转换回原始词汇。

5. **`self.n_words`**:
   - 词汇表大小，即词汇数量。通过计算 `self.decoder` 的长度获得。

6. **初始化过程**：
   - 读取词汇文件，解析每一行以获取词汇及其排名，然后解码词汇并存储到 `mergeable_ranks` 字典中。接着，使用 `tiktoken` 库初始化编码器，并创建解码器和词汇表大小。

其中有一点值得关注：
```python
    def convert_tokens_to_string(self, tokens: List[Union[bytes, str]]) -> str:
        """
        将 tokens 序列转换为字符串。
        """
        text = ""
        temp = b""
        for t in tokens:
            if isinstance(t, str):
                if temp:
                    text += temp.decode("utf-8", errors="replace")
                    temp = b""
                text += t
            elif isinstance(t, bytes):
                temp += t
            else:
                raise TypeError("token should only be of type bytes or str")
        if temp:
            text += temp.decode("utf-8", errors="replace")
        return text
```

这个函数之所以要将 `token` 解码为 `bytes` 是因为在实际应用中，token 可能是字符串或者字节序列的一部分。在某些情况下，词汇表中的 token 可能以 `bytes` 的形式存储，而不是直接存储为字符串。以下是一些详细原因和场景：

1. **支持多种编码格式**：
   - 分词器可能处理多种数据源，其中一些数据源可能以 `bytes` 格式存储 token。例如，当数据被压缩、加密或使用特定编码时，token 可能会以字节形式表示。

2. **数据处理的灵活性**：
   - 通过支持 `bytes` 和 `str` 两种类型，分词器可以更灵活地处理不同来源和格式的输入数据。这在处理包含二进制数据的文本时特别有用，例如一些特殊的标记或字符。

3. **确保数据一致性**：
   - 在分词和解码过程中，可能会遇到混合类型的 token（即既有字符串也有字节序列）。为了确保数据一致性并正确拼接字符串，函数需要处理 `bytes` 和 `str` 两种类型。

4. **兼容性考虑**：
   - 某些 NLP 工具和库在处理文本时可能会返回字节形式的 token。为了与这些工具和库兼容，分词器需要能够处理和转换这些字节 token。

具体来看这个函数的工作流程：

1. **初始化空字符串和字节序列**：
   - `text = ""` 初始化一个空字符串，用于存储最终的结果。
   - `temp = b""` 初始化一个空字节序列，用于临时存储字节 token。

2. **遍历 token 列表**：
   - 对于每个 token，检查其类型。
     - 如果 token 是字符串类型（`str`），检查 `temp` 是否为空。如果 `temp` 非空，将 `temp` 解码为字符串并添加到 `text`，然后清空 `temp`。之后，将当前字符串 token 添加到 `text`。
     - 如果 token 是字节类型（`bytes`），将其添加到 `temp`，以便后续解码。
     - 如果 token 既不是字符串也不是字节类型，抛出类型错误。

3. **处理剩余的字节序列**：
   - 在循环结束后，如果 `temp` 非空，将其解码为字符串并添加到 `text`。

通过上述步骤，函数能够正确处理混合类型的 token 列表，并将其转换为一个完整的字符串。以下是代码中的注释，以更好地解释这些步骤：

```python
def convert_tokens_to_string(self, tokens: List[Union[bytes, str]]) -> str:
    """
    将 tokens 序列转换为字符串。
    """
    text = ""  # 初始化一个空字符串用于存储结果
    temp = b""  # 初始化一个空字节序列用于临时存储字节 token
    for t in tokens:
        if isinstance(t, str):
            if temp:
                # 如果 temp 非空，将其解码为字符串并添加到 text 中
                text += temp.decode("utf-8", errors="replace")
                temp = b""  # 清空 temp
            text += t  # 将字符串 token 添加到 text 中
        elif isinstance(t, bytes):
            temp += t  # 将字节 token 添加到 temp 中
        else:
            raise TypeError("token should only be of type bytes or str")  # 抛出类型错误
    if temp:
        # 处理剩余的字节序列，将其解码为字符串并添加到 text 中
        text += temp.decode("utf-8", errors="replace")
    return text  # 返回最终的字符串
```

接下来设置config，简单设置相关属性即可。

In [62]:
from transformers import PretrainedConfig


class ChatGLMConfig(PretrainedConfig):
    model_type = "chatglm"

    def __init__(
            self,
            num_layers=28,
            padded_vocab_size=65024,
            hidden_size=4096,
            ffn_hidden_size=13696,
            kv_channels=128,
            num_attention_heads=32,
            seq_length=2048,
            hidden_dropout=0.0,
            classifier_dropout=None,
            attention_dropout=0.0,
            layernorm_epsilon=1e-5,
            rmsnorm=True,
            apply_residual_connection_post_layernorm=False,
            post_layer_norm=True,
            add_bias_linear=False,
            add_qkv_bias=False,
            bias_dropout_fusion=True,
            multi_query_attention=False,
            multi_query_group_num=1,
            rope_ratio=1,
            apply_query_key_layer_scaling=True,
            attention_softmax_in_fp32=True,
            fp32_residual_connection=False,
            **kwargs
    ):
        self.num_layers = num_layers
        self.vocab_size = padded_vocab_size
        self.padded_vocab_size = padded_vocab_size
        self.hidden_size = hidden_size
        self.ffn_hidden_size = ffn_hidden_size
        self.kv_channels = kv_channels
        self.num_attention_heads = num_attention_heads
        self.seq_length = seq_length
        self.hidden_dropout = hidden_dropout
        self.classifier_dropout = classifier_dropout
        self.attention_dropout = attention_dropout
        self.layernorm_epsilon = layernorm_epsilon
        self.rmsnorm = rmsnorm
        self.apply_residual_connection_post_layernorm = apply_residual_connection_post_layernorm
        self.post_layer_norm = post_layer_norm
        self.add_bias_linear = add_bias_linear
        self.add_qkv_bias = add_qkv_bias
        self.bias_dropout_fusion = bias_dropout_fusion
        self.multi_query_attention = multi_query_attention
        self.multi_query_group_num = multi_query_group_num
        self.rope_ratio = rope_ratio
        self.apply_query_key_layer_scaling = apply_query_key_layer_scaling
        self.attention_softmax_in_fp32 = attention_softmax_in_fp32
        self.fp32_residual_connection = fp32_residual_connection
        super().__init__(**kwargs)


然后就可以开始构建我们的模型了。

In [63]:
import json
import math
import copy
import warnings
import re
import sys

import torch
import torch.utils.checkpoint
import torch.nn.functional as F
from torch import nn
from torch.nn import CrossEntropyLoss, LayerNorm, MSELoss, BCEWithLogitsLoss
from torch.nn.utils import skip_init
from typing import Optional, Tuple, Union, List, Callable, Dict, Any
from copy import deepcopy

from transformers.modeling_outputs import (
    BaseModelOutputWithPast,
    CausalLMOutputWithPast,
    SequenceClassifierOutputWithPast,
)
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import logging
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList, GenerationConfig, ModelOutput

In [64]:
sys.platform

'linux'

In [65]:
# 如果系统平台不是 Darwin (即 macOS或 linux)，则进行以下设置以提高性能
if sys.platform != 'darwin':
    # 关闭 JIT 的 profiling 模式
    torch._C._jit_set_profiling_mode(False)
    # 关闭 JIT 的 profiling 执行器
    torch._C._jit_set_profiling_executor(False)
    # 允许在 CPU 上进行张量融合
    torch._C._jit_override_can_fuse_on_cpu(True)
    # 允许在 GPU 上进行张量融合
    torch._C._jit_override_can_fuse_on_gpu(True)

# 获取日志记录器
logger = logging.get_logger(__name__)

# 用于文档的检查点
_CHECKPOINT_FOR_DOC = "THUDM/ChatGLM"
# 用于文档的配置
_CONFIG_FOR_DOC = "ChatGLMConfig"

# 默认初始化函数
def default_init(cls, *args, **kwargs):
    return cls(*args, **kwargs)

# 无效分数 Logits 处理器类
class InvalidScoreLogitsProcessor(LogitsProcessor):
    # 处理输入的 logits
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        # 如果 logits 中存在 NaN 或无穷大，则将其重置
        if torch.isnan(scores).any() or torch.isinf(scores).any():
            scores.zero_()  # 将所有分数重置为 0
            scores[..., 198] = 5e4  # 将特定位置的分数设置为一个大值
        return scores

# 沿着最后一个维度分割张量的函数
def split_tensor_along_last_dim(
        tensor: torch.Tensor,
        num_partitions: int,
        contiguous_split_chunks: bool = False,
) -> List[torch.Tensor]:
    """
    沿着最后一个维度分割张量。

    参数:
        tensor: 输入张量。
        num_partitions: 要分割的部分数量。
        contiguous_split_chunks: 如果为 True，则使每个分块在内存中是连续的。

    返回:
        张量的列表
    """
    # 获取最后一个维度的大小
    last_dim = tensor.dim() - 1
    last_dim_size = tensor.size()[last_dim] // num_partitions
    # 分割张量
    tensor_list = torch.split(tensor, last_dim_size, dim=last_dim)
    # 注意: torch.split 默认不会创建连续的张量
    if contiguous_split_chunks:
        return tuple(chunk.contiguous() for chunk in tensor_list)
    return tensor_list


### 关键点解释

1. **系统平台检查**：
   - 如果系统平台不是 `Darwin`（macOS），则关闭 JIT 的 profiling 模式和执行器，并启用 CPU 和 GPU 上的张量融合。这些设置可以提高模型的性能。

2. **日志记录器**：
   - 获取一个日志记录器，用于记录模型构建过程中的信息和调试。

3. **文档字符串**：
   - `_CHECKPOINT_FOR_DOC` 和 `_CONFIG_FOR_DOC` 是用于文档生成的字符串常量，指示模型和配置的检查点和配置文件。

4. **默认初始化函数**：
   - `default_init` 函数是一个通用的初始化函数，用于实例化类。

5. **`InvalidScoreLogitsProcessor` 类**：
   - 这是一个 logits 处理器类，用于处理无效的 logits 分数。如果分数中存在 `NaN` 或无穷大值，将这些值重置为 0，并将特定位置（例如 198）的分数设置为一个很大的值（`5e4`），以确保模型输出有效的结果。

6. **`split_tensor_along_last_dim` 函数**：
   - 这个函数用于沿着张量的最后一个维度分割张量。参数包括输入张量、要分割的部分数量和一个布尔值（是否使每个分块在内存中是连续的）。如果 `contiguous_split_chunks` 为真，则每个分块在内存中是连续的；否则，直接返回分割后的张量列表。


In [66]:
# 旋转位置嵌入类
class RotaryEmbedding(nn.Module):
    def __init__(self, dim, rope_ratio=1, original_impl=False, device=None, dtype=None):
        super().__init__()
        # 计算倒数频率
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2, device=device).to(dtype=dtype) / dim))
        # 注册倒数频率为 buffer
        self.register_buffer("inv_freq", inv_freq)
        self.dim = dim
        self.original_impl = original_impl
        self.rope_ratio = rope_ratio

    # 实现前向传播的具体方法
    def forward_impl(
            self, seq_len: int, n_elem: int, dtype: torch.dtype, device: torch.device, base: int = 10000
    ):
        """
        增强的 Transformer 使用旋转位置嵌入。

        参考自:
        https://github.com/labmlai/annotated_deep_learning_paper_implementations/blob/master/labml_nn/
        transformers/rope/__init__.py. MIT 许可证:
        https://github.com/labmlai/annotated_deep_learning_paper_implementations/blob/master/license.
        """
        # 计算 $\Theta = {\theta_i = 10000^{\frac{2(i-1)}{d}}, i \in [1, 2, ..., \frac{d}{2}]}$
        base = base * self.rope_ratio
        theta = 1.0 / (base ** (torch.arange(0, n_elem, 2, dtype=torch.float, device=device) / n_elem))

        # 创建位置索引 `[0, 1, ..., seq_len - 1]`
        seq_idx = torch.arange(seq_len, dtype=torch.float, device=device)

        # 计算位置索引和 $\theta_i$ 的乘积
        idx_theta = torch.outer(seq_idx, theta).float()

        # 缓存计算的余弦和正弦值
        cache = torch.stack([torch.cos(idx_theta), torch.sin(idx_theta)], dim=-1)

        # 模拟 complex32 的行为，否则会得到不同的结果
        if dtype in (torch.float16, torch.bfloat16, torch.int8):
            cache = cache.bfloat16() if dtype == torch.bfloat16 else cache.half()
        return cache

    # 前向传播方法
    def forward(self, max_seq_len, offset=0):
        return self.forward_impl(
            max_seq_len, self.dim, dtype=self.inv_freq.dtype, device=self.inv_freq.device
        )

# 使用 TorchScript JIT 编译器优化的旋转位置嵌入应用函数
@torch.jit.script
def apply_rotary_pos_emb(x: torch.Tensor, rope_cache: torch.Tensor) -> torch.Tensor:
    # x: [b, np, sq, hn]
    b, np, sq, hn = x.size(0), x.size(1), x.size(2), x.size(3)
    rot_dim = rope_cache.shape[-2] * 2
    x, x_pass = x[..., :rot_dim], x[..., rot_dim:]
    # 截断以支持可变大小
    rope_cache = rope_cache[:, :sq]
    xshaped = x.reshape(b, np, sq, rot_dim // 2, 2)
    rope_cache = rope_cache.view(-1, 1, sq, xshaped.size(3), 2)
    x_out2 = torch.stack(
        [
            xshaped[..., 0] * rope_cache[..., 0] - xshaped[..., 1] * rope_cache[..., 1],
            xshaped[..., 1] * rope_cache[..., 0] + xshaped[..., 0] * rope_cache[..., 1],
        ],
        -1,
    )
    x_out2 = x_out2.flatten(3)
    return torch.cat((x_out2, x_pass), dim=-1)

# RMS 归一化层类
class RMSNorm(torch.nn.Module):
    def __init__(self, normalized_shape, eps=1e-5, device=None, dtype=None, **kwargs):
        super().__init__()
        # 初始化权重参数
        self.weight = torch.nn.Parameter(torch.empty(normalized_shape, device=device, dtype=dtype))
        self.eps = eps

    def forward(self, hidden_states: torch.Tensor):
        # 获取输入张量的数据类型
        input_dtype = hidden_states.dtype
        # 计算方差
        variance = hidden_states.to(torch.float32).pow(2).mean(-1, keepdim=True)
        # 进行 RMS 归一化
        hidden_states = hidden_states * torch.rsqrt(variance + self.eps)
        # 应用权重并返回与输入相同的数据类型
        return (self.weight * hidden_states).to(input_dtype)

### 关键点解释

1. **`RotaryEmbedding` 类**：
   - 用于实现旋转位置嵌入的模块。
   - `inv_freq` 是一个倒数频率的张量，用于位置嵌入的计算。
   - `forward_impl` 方法根据序列长度和嵌入维度计算旋转位置嵌入。
   - `forward` 方法调用 `forward_impl` 进行前向传播。

2. **`apply_rotary_pos_emb` 函数**：
   - 使用旋转位置嵌入更新输入张量。
   - `x` 是输入张量，`rope_cache` 是预计算的旋转位置嵌入。
   - 该函数首先对输入张量进行分割和重塑，然后将旋转位置嵌入应用于每个分块，最后将结果拼接回原始张量。

3. **`RMSNorm` 类**：
   - 实现 RMS 归一化的模块。
   - `weight` 是归一化的权重参数。
   - `forward` 方法计算输入张量的方差，并进行归一化处理，然后应用权重。

这些注释和解释可以帮助理解每个部分的功能和实现细节，对于模型构建和调试非常有用。

In [67]:
class CoreAttention(torch.nn.Module):
    def __init__(self, config: ChatGLMConfig, layer_number):
        super(CoreAttention, self).__init__()

        # 配置参数
        self.apply_query_key_layer_scaling = config.apply_query_key_layer_scaling
        self.attention_softmax_in_fp32 = config.attention_softmax_in_fp32
        if self.apply_query_key_layer_scaling:
            self.attention_softmax_in_fp32 = True
        self.layer_number = max(1, layer_number)

        # 计算投影大小
        projection_size = config.kv_channels * config.num_attention_heads

        # 每个注意力头和每个分区的值
        self.hidden_size_per_partition = projection_size
        self.hidden_size_per_attention_head = projection_size // config.num_attention_heads
        self.num_attention_heads_per_partition = config.num_attention_heads

        coeff = None
        self.norm_factor = math.sqrt(self.hidden_size_per_attention_head)
        if self.apply_query_key_layer_scaling:
            coeff = self.layer_number
            self.norm_factor *= coeff
        self.coeff = coeff

        # 注意力 dropout
        self.attention_dropout = torch.nn.Dropout(config.attention_dropout)

    def forward(self, query_layer, key_layer, value_layer, attention_mask):
        pytorch_major_version = int(torch.__version__.split('.')[0])
        if pytorch_major_version >= 2:
            # PyTorch 2.0 及以上版本
            if attention_mask is None and query_layer.shape[2] == key_layer.shape[2]:
                context_layer = torch.nn.functional.scaled_dot_product_attention(query_layer, key_layer, value_layer,
                                                                                 is_causal=True)
            else:
                if attention_mask is not None:
                    attention_mask = ~attention_mask
                context_layer = torch.nn.functional.scaled_dot_product_attention(query_layer, key_layer, value_layer,
                                                                                 attention_mask)
            context_layer = context_layer.transpose(1, 2).contiguous()
            new_context_layer_shape = context_layer.size()[:-2] + (self.hidden_size_per_partition,)
            context_layer = context_layer.reshape(*new_context_layer_shape)
        else:
            # 处理 PyTorch 2.0 以下版本

            # 原始注意力得分
            # [b, np, sq, sk]
            output_size = (query_layer.size(0), query_layer.size(1), query_layer.size(2), key_layer.size(2))

            # 重新调整视图 [b, np, sq, hn] -> [b * np, sq, hn]
            query_layer = query_layer.view(output_size[0] * output_size[1], output_size[2], -1)
            # 重新调整视图 [b, np, sk, hn] -> [b * np, sk, hn]
            key_layer = key_layer.view(output_size[0] * output_size[1], output_size[3], -1)

            # 预分配输入张量: [b * np, sq, sk]
            matmul_input_buffer = torch.empty(
                output_size[0] * output_size[1], output_size[2], output_size[3], dtype=query_layer.dtype,
                device=query_layer.device
            )

            # 计算原始注意力得分. [b * np, sq, sk]
            matmul_result = torch.baddbmm(
                matmul_input_buffer,
                query_layer,  # [b * np, sq, hn]
                key_layer.transpose(1, 2),  # [b * np, hn, sk]
                beta=0.0,
                alpha=(1.0 / self.norm_factor),
            )

            # 改变视图到 [b, np, sq, sk]
            attention_scores = matmul_result.view(*output_size)

            # 处理注意力得分和 dropout
            if self.attention_softmax_in_fp32:
                attention_scores = attention_scores.float()
            if self.coeff is not None:
                attention_scores = attention_scores * self.coeff
            if attention_mask is None and attention_scores.shape[2] == attention_scores.shape[3]:
                attention_mask = torch.ones(output_size[0], 1, output_size[2], output_size[3],
                                            device=attention_scores.device, dtype=torch.bool)
                attention_mask.tril_()
                attention_mask = ~attention_mask
            if attention_mask is not None:
                attention_scores = attention_scores.masked_fill(attention_mask, float("-inf"))
            attention_probs = F.softmax(attention_scores, dim=-1)
            attention_probs = attention_probs.type_as(value_layer)

            # 丢弃整个 token 的注意力，这源自原始的 Transformer 论文
            attention_probs = self.attention_dropout(attention_probs)

            # 重新调整视图 [b * np, sq, hn]
            value_layer = value_layer.view(output_size[0] * output_size[1], value_layer.size(2), -1)
            # 重新调整视图 [b * np, sq, sk]
            attention_probs = attention_probs.view(output_size[0] * output_size[1], output_size[2], -1)
            # 计算上下文层 [b * np, sq, hn]
            context_layer = torch.bmm(attention_probs, value_layer)
            # 重新调整视图 [b, np, sq, hn]
            context_layer = context_layer.view(*output_size)
            # [b, np, sq, hn] --> [b, sq, np, hn]
            context_layer = context_layer.transpose(1, 2).contiguous()
            # [b, sq, np, hn] --> [b, sq, hp]
            new_context_layer_shape = context_layer.size()[:-2] + (self.hidden_size_per_partition,)
            context_layer = context_layer.reshape(*new_context_layer_shape)

        return context_layer

### CoreAttention 详解及公式

在神经网络模型中，特别是 Transformer 架构中，注意力机制起着至关重要的作用。CoreAttention 实现了自注意力机制中的核心部分，具体来说包括以下步骤：计算注意力得分、应用注意力掩码、计算注意力权重以及计算上下文向量。

#### 1. 计算注意力得分 (Attention Scores)

注意力得分的计算可以表示为矩阵乘法。对于查询 (query) 向量 $Q$ 和键 (key) 向量 $K$，计算注意力得分矩阵 $A$ 的公式为：
\begin{align*} A = \frac{QK^T}{\sqrt{d_k}} \end{align*}
其中 $d_k$ 是键向量的维度，这里的 $\sqrt{d_k}$ 是一个缩放因子，防止得分值过大。

在代码中，通过矩阵乘法实现：
```python
matmul_result = torch.baddbmm(
    matmul_input_buffer,
    query_layer,  # [b * np, sq, hn]
    key_layer.transpose(1, 2),  # [b * np, hn, sk]
    beta=0.0,
    alpha=(1.0 / self.norm_factor),
)
```
其中 `self.norm_factor` 是 $\sqrt{d_k}$。

#### 2. 应用注意力掩码 (Attention Mask)

为了避免模型关注不必要的部分，使用注意力掩码来屏蔽某些位置。注意力掩码通过将相应位置的注意力得分设为负无穷大来实现：
```python
if attention_mask is not None:
    attention_scores = attention_scores.masked_fill(attention_mask, float("-inf"))
```

#### 3. 计算注意力权重 (Attention Weights)

应用 softmax 函数将注意力得分转换为概率分布，表示每个查询向量对键向量的关注程度：
\begin{align*} \text{AttentionWeights} = \text{softmax}(A) \end{align*}
代码中实现为：
```python
attention_probs = F.softmax(attention_scores, dim=-1)
attention_probs = attention_probs.type_as(value_layer)
```

#### 4. 计算上下文向量 (Context Vectors)

上下文向量通过将注意力权重与值 (value) 向量相乘并求和得到：
\begin{align*} \text{Context} = \text{AttentionWeights} \cdot V \end{align*}
其中 $V$ 是值向量。

在代码中实现为：
```python
context_layer = torch.bmm(attention_probs, value_layer)
```

#### 详细原理解释

1. **初始化和设置**：
    - 初始化类时，会根据配置参数设置注意力缩放、层数等信息。
    - `projection_size` 定义了投影大小，`hidden_size_per_attention_head` 和 `num_attention_heads_per_partition` 定义了每个注意力头和每个分区的隐藏层大小。

2. **前向传播步骤**：
    - **查询、键和值的计算**：计算查询、键和值向量。
    - **计算注意力得分**：通过矩阵乘法计算注意力得分矩阵。
    - **应用注意力掩码**：将需要屏蔽的位置设置为负无穷大，避免影响后续计算。
    - **计算注意力权重**：应用 softmax 函数，得到注意力权重。
    - **计算上下文向量**：将注意力权重与值向量相乘，得到上下文向量。

3. **具体实现细节**：
    - 根据 PyTorch 版本，选择合适的注意力计算方式。
    - 对不同维度的张量进行变换，确保形状匹配。
    - 使用 dropout 防止过拟合。

### 公式和代码对应关系

- **注意力得分**：
  \begin{align*}
  A = \frac{QK^T}{\sqrt{d_k}}
  \end{align*}
  对应代码：
  ```python
  matmul_result = torch.baddbmm(
      matmul_input_buffer,
      query_layer,
      key_layer.transpose(1, 2),
      beta=0.0,
      alpha=(1.0 / self.norm_factor),
  )
  ```

- **应用注意力掩码**：
  \begin{align*}
  A'_{ij} = \begin{cases} 
  A_{ij} & \text{if } \text{mask}_{ij} = 1 \\
  -\infty & \text{if } \text{mask}_{ij} = 0 
  \end{cases}
  \end{align*}
  对应代码：
  ```python
  if attention_mask is not None:
      attention_scores = attention_scores.masked_fill(attention_mask, float("-inf"))
  ```

- **注意力权重**：
  \begin{align*}
  \text{AttentionWeights}_{ij} = \frac{\exp(A'_{ij})}{\sum_k \exp(A'_{ik})}
  \end{align*}
  对应代码：
  ```python
  attention_probs = F.softmax(attention_scores, dim=-1)
  ```

- **上下文向量**：
  \begin{align*}
  \text{Context}_{i} = \sum_j \text{AttentionWeights}_{ij} V_j
  \end{align*}
  对应代码：
  ```python
  context_layer = torch.bmm(attention_probs, value_layer)
  ```


接下来，我们再重温一下注意力机制

### 注意力机制原理及公式解释

#### 1. 注意力机制是什么？
注意力机制（Attention Mechanism）是深度学习中特别是自然语言处理领域中的一种技术，它使模型能够在处理输入序列时动态地关注不同的部分。简单来说，注意力机制让模型在处理某个元素时，可以有选择地关注输入序列的其他部分，而不是全部一视同仁。

#### 2. 注意力机制的核心公式
我们来看注意力得分的公式：
\begin{align*} A = \frac{QK^T}{\sqrt{d_k}} \end{align*}

这里的符号解释如下：
- \( Q \) 是查询（Query）向量。
- \( K \) 是键（Key）向量。
- \( d_k \) 是键向量的维度。
- \( A \) 是注意力得分矩阵。

#### 3. 为什么使用这个公式计算注意力得分？

注意力得分公式的核心思想是通过计算查询向量和键向量的点积，来衡量查询与每个键之间的相关性。点积结果越大，表示查询与该键的相关性越强，模型应该对该键对应的值（Value）向量给予更多的关注。

公式中的缩放因子 \(\sqrt{d_k}\) 是为了避免点积结果过大，因为如果不缩放，较大的向量维度会导致点积结果非常大，进而导致 softmax 函数输出接近于零的梯度，影响模型训练效果。

#### 4. 计算注意力权重
得到注意力得分矩阵 \( A \) 后，通过 softmax 函数将其转换为注意力权重矩阵：
\begin{align*} \text{AttentionWeights}_{ij} = \frac{\exp(A'_{ij})}{\sum_k \exp(A'_{ik})} \end{align*}
其中 \( A' \) 是应用掩码后的注意力得分矩阵。

#### 5. 计算上下文向量
最后，使用注意力权重矩阵对值向量进行加权求和，得到上下文向量：
\begin{align*} \text{Context}_{i} = \sum_j \text{AttentionWeights}_{ij} V_j \end{align*}
这里 \( V \) 是值向量。

### 注意力机制与人类注意力的关系

注意力机制与人类的注意力有一定的相似之处，但也有显著的区别。

- **相似之处**：
  - **选择性关注**：就像人类在阅读一篇文章时会选择性地关注某些重要段落，忽略其他部分，注意力机制也让模型在处理一个序列时可以选择性地关注不同的部分。
  - **动态调整**：人类的注意力是动态的，会根据上下文调整关注点。注意力机制也是动态的，可以根据输入的变化调整注意力权重。

- **区别**：
  - **机制不同**：人类注意力是通过大脑的复杂神经网络实现的，包括视觉、听觉等多种感官信息的综合处理。而注意力机制是一种数学方法，通过点积、softmax 等操作实现。
  - **目的不同**：人类注意力用于理解和互动，而注意力机制主要用于提高模型在处理长序列数据时的性能和效率。

### 具体代码实现中的细节

在 `CoreAttention` 类中，注意力得分的计算和应用通过以下代码片段实现：
```python
matmul_result = torch.baddbmm(
    matmul_input_buffer,
    query_layer,  # [b * np, sq, hn]
    key_layer.transpose(1, 2),  # [b * np, hn, sk]
    beta=0.0,
    alpha=(1.0 / self.norm_factor),
)
```
这里 `torch.baddbmm` 函数执行的是批量矩阵乘法，计算公式中的 \(QK^T\) 部分，并除以 \(\sqrt{d_k}\) 进行缩放。

应用 softmax 得到注意力权重：
```python
attention_probs = F.softmax(attention_scores, dim=-1)
```

最后，计算上下文向量：
```python
context_layer = torch.bmm(attention_probs, value_layer)
```

这一步将注意力权重与值向量相乘并求和，得到最终的上下文表示。

通过上述解释，可以更好地理解注意力机制的原理和实现，以及它在模型中的重要作用。

完成注意力机制的核心实现后，我们构建自注意力

### SelfAttention 类的原理及公式解释

#### SelfAttention 类的功能

SelfAttention 类实现了自注意力机制（Self-Attention Mechanism），这是 Transformer 模型的核心部分。自注意力机制的目标是让每个位置的表示能够动态地关注输入序列的其他位置，从而捕捉全局信息。

#### 自注意力机制的步骤及公式

自注意力机制包括以下几个关键步骤：

1. **线性变换**：
   输入的隐藏状态 \(X\) 通过线性层分别映射到查询 \(Q\)、键 \(K\) 和值 \(V\) 三个空间：
   \begin{align*}
   Q = XW_Q, \quad K = XW_K, \quad V = XW_V
   \end{align*}
   其中，\(W_Q\)、\(W_K\) 和 \(W_V\) 是学习到的权重矩阵。

2. **计算注意力得分**：
   通过计算查询 \(Q\) 和键 \(K\) 的点积并除以缩放因子 \(\sqrt{d_k}\) 得到注意力得分矩阵 \(A\)：
   \begin{align*}
   A = \frac{QK^T}{\sqrt{d_k}}
   \end{align*}

3. **应用注意力掩码**：
   对于自注意力机制，如果使用掩码（例如在解码阶段），将不需要关注的位置设为负无穷大以屏蔽：
   \begin{align*}
   A'_{ij} = \begin{cases} 
   A_{ij} & \text{if } \text{mask}_{ij} = 1 \\
   -\infty & \text{if } \text{mask}_{ij} = 0 
   \end{cases}
   \end{align*}

4. **计算注意力权重**：
   对注意力得分矩阵 \(A'\) 应用 softmax 函数，得到注意力权重矩阵：
   \begin{align*}
   \text{AttentionWeights}_{ij} = \frac{\exp(A'_{ij})}{\sum_k \exp(A'_{ik})}
   \end{align*}

5. **计算上下文向量**：
   使用注意力权重对值 \(V\) 进行加权求和，得到上下文向量：
   \begin{align*}
   \text{Context} = \text{AttentionWeights} \cdot V
   \end{align*}

### SelfAttention 类的具体实现

以下是 SelfAttention 类中的核心步骤和公式的实现细节：



In [68]:
class SelfAttention(torch.nn.Module):
    """并行自注意力层的抽象类。

    自注意力层接受形状为 [s, b, h] 的输入，并返回相同形状的输出。
    """

    def __init__(self, config: ChatGLMConfig, layer_number, device=None):
        super(SelfAttention, self).__init__()
        self.layer_number = max(1, layer_number)

        self.projection_size = config.kv_channels * config.num_attention_heads

        # 每个注意力头和每个分区的值
        self.hidden_size_per_attention_head = self.projection_size // config.num_attention_heads
        self.num_attention_heads_per_partition = config.num_attention_heads

        self.multi_query_attention = config.multi_query_attention
        self.qkv_hidden_size = 3 * self.projection_size
        if self.multi_query_attention:
            self.num_multi_query_groups_per_partition = config.multi_query_group_num
            self.qkv_hidden_size = (
                    self.projection_size + 2 * self.hidden_size_per_attention_head * config.multi_query_group_num
            )
        self.query_key_value = nn.Linear(config.hidden_size, self.qkv_hidden_size,
                                         bias=config.add_bias_linear or config.add_qkv_bias,
                                         device=device, **_config_to_kwargs(config)
                                         )

        self.core_attention = CoreAttention(config, self.layer_number)

        # 输出层
        self.dense = nn.Linear(self.projection_size, config.hidden_size, bias=config.add_bias_linear,
                               device=device, **_config_to_kwargs(config)
                               )

    def forward(self, hidden_states, attention_mask, rotary_pos_emb, kv_cache=None, use_cache=True):
        # hidden_states: [b, sq, h]

        # =====================
        # Query, Key 和 Value
        # =====================

        # 注意力头 [b, sq, h] --> [b, sq, (np * 3 * hn)]
        mixed_x_layer = self.query_key_value(hidden_states)

        if self.multi_query_attention:
            (query_layer, key_layer, value_layer) = mixed_x_layer.split(
                [
                    self.num_attention_heads_per_partition * self.hidden_size_per_attention_head,
                    self.num_multi_query_groups_per_partition * self.hidden_size_per_attention_head,
                    self.num_multi_query_groups_per_partition * self.hidden_size_per_attention_head,
                ],
                dim=-1,
            )
            query_layer = query_layer.view(
                query_layer.size()[:-1] + (self.num_attention_heads_per_partition, self.hidden_size_per_attention_head)
            )
            key_layer = key_layer.view(
                key_layer.size()[:-1] + (self.num_multi_query_groups_per_partition, self.hidden_size_per_attention_head)
            )
            value_layer = value_layer.view(
                value_layer.size()[:-1]
                + (self.num_multi_query_groups_per_partition, self.hidden_size_per_attention_head)
            )
        else:
            new_tensor_shape = mixed_x_layer.size()[:-1] + \
                               (self.num_attention_heads_per_partition,
                                3 * self.hidden_size_per_attention_head)
            mixed_x_layer = mixed_x_layer.view(*new_tensor_shape)

            # [b, sq, np, 3 * hn] --> 3 [b, sq, np, hn]
            (query_layer, key_layer, value_layer) = split_tensor_along_last_dim(mixed_x_layer, 3)

        # [b, sq, np, hn] -> [b, np, sq, hn]
        query_layer, key_layer, value_layer = [k.transpose(1, 2) for k in [query_layer, key_layer, value_layer]]

        # 应用相对位置编码（旋转嵌入）
        if rotary_pos_emb is not None:
            query_layer = apply_rotary_pos_emb(query_layer, rotary_pos_emb)
            key_layer = apply_rotary_pos_emb(key_layer, rotary_pos_emb)

        # 调整 key 和 value 以用于推理
        if kv_cache is not None:
            cache_k, cache_v = kv_cache
            key_layer = torch.cat((cache_k, key_layer), dim=2)
            value_layer = torch.cat((cache_v, value_layer), dim=2)
        if use_cache:
            if kv_cache is None:
                kv_cache = torch.cat((key_layer.unsqueeze(0).unsqueeze(0), value_layer.unsqueeze(0).unsqueeze(0)), dim=1)
            else:
                kv_cache = (key_layer, value_layer)
        else:
            kv_cache = None

        if self.multi_query_attention:
            key_layer = key_layer.unsqueeze(2)
            key_layer = key_layer.expand(
                -1, -1, self.num_attention_heads_per_partition // self.num_multi_query_groups_per_partition, -1, -1
            )
            key_layer = key_layer.contiguous().view(
                key_layer.size()[:1] + (self.num_attention_heads_per_partition,) + key_layer.size()[3:]
            )
            value_layer = value_layer.unsqueeze(2)
            value_layer = value_layer.expand(
                -1, -1, self.num_attention_heads_per_partition // self.num_multi_query_groups_per_partition, -1, -1
            )
            value_layer = value_layer.contiguous().view(
                value_layer.size()[:1] + (self.num_attention_heads_per_partition,) + value_layer.size()[3:]
            )

        # 核心注意力计算
        context_layer = self.core_attention(query_layer, key_layer, value_layer, attention_mask)

        # 输出. [sq, b, h]
        output = self.dense(context_layer)

        return output, kv_cache

### 详细原理解释

1. **线性变换**：
   ```python
   mixed_x_layer = self.query_key_value(hidden_states)
   ```
   输入隐藏状态 `hidden_states` 通过线性层映射到查询、键和值的空间。

2. **拆分查询、键和值**：
   ```python
   (query_layer, key_layer, value_layer) = split_tensor_along_last_dim(mixed_x_layer, 3)
   ```
   将线性变换后的结果拆分成查询、键和值。

3. **形状变换**：
   ```python
   query_layer, key_layer, value_layer = [k.transpose(1, 2) for k in [query_layer, key_layer, value_layer]]
   ```
   将查询、键和值的形状从 `[b, sq, np, hn]` 转换为 `[b, np, sq, hn]`。

4. **应用旋转位置编码**：
   ```python
   if rotary_pos_emb is not None:
       query_layer = apply_rotary_pos_emb(query_layer, rotary_pos_emb)
       key_layer = apply_rotary_pos_emb(key_layer, rotary_pos_emb)
   ```
   如果存在旋转位置编码，则对查询和键应用位置编码。

5. **调整键和值用于推理**：
   ```python
   if kv_cache is not None:
       cache_k, cache_v = kv_cache
      

 key_layer = torch.cat((cache_k, key_layer), dim=2)
       value_layer = torch.cat((cache_v, value_layer), dim=2)
   if use_cache:
       if kv_cache is None:
           kv_cache = torch.cat((key_layer.unsqueeze(0).unsqueeze(0), value_layer.unsqueeze(0).unsqueeze(0)), dim=1)
       else:
           kv_cache = (key_layer, value_layer)
   else:
       kv_cache = None
   ```

6. **核心注意力计算**：
   ```python
   context_layer = self.core_attention(query_layer, key_layer, value_layer, attention_mask)
   ```

7. **输出层**：
   ```python
   output = self.dense(context_layer)
   ```

通过上述详细的原理解释和公式，可以更好地理解 SelfAttention 类的实现以及其在 Transformer 模型中的作用。自注意力机制通过动态调整不同位置之间的权重，使得模型能够更有效地捕捉全局信息，从而提高模型的性能和泛化能力。

In [69]:
def _config_to_kwargs(args):
    common_kwargs = {
        "dtype": args.torch_dtype,
    }
    return common_kwargs


class MLP(torch.nn.Module):
    """多层感知机（MLP）。

    MLP 将接受隐藏状态为 h 的输入，将其投影到 4*h 的隐藏维度，执行非线性变换，然后将状态投影回 h 的隐藏维度。
    """

    def __init__(self, config: ChatGLMConfig, device=None):
        super(MLP, self).__init__()

        self.add_bias = config.add_bias_linear

        # 投影到 4h。如果使用 swiglu 则将输出宽度加倍，详见 https://arxiv.org/pdf/2002.05202.pdf
        self.dense_h_to_4h = nn.Linear(
            config.hidden_size,
            config.ffn_hidden_size * 2,
            bias=self.add_bias,
            device=device,
            **_config_to_kwargs(config)
        )

        def swiglu(x):
            x = torch.chunk(x, 2, dim=-1)
            return F.silu(x[0]) * x[1]

        self.activation_func = swiglu

        # 投影回 h.
        self.dense_4h_to_h = nn.Linear(
            config.ffn_hidden_size,
            config.hidden_size,
            bias=self.add_bias,
            device=device,
            **_config_to_kwargs(config)
        )

    def forward(self, hidden_states):
        # [s, b, 4hp]
        intermediate_parallel = self.dense_h_to_4h(hidden_states)
        intermediate_parallel = self.activation_func(intermediate_parallel)
        # [s, b, h]
        output = self.dense_4h_to_h(intermediate_parallel)
        return output


class GLMBlock(torch.nn.Module):
    """一个 transformer 层。

    Transformer 层接受形状为 [s, b, h] 的输入，并返回相同形状的输出。
    """

    def __init__(self, config: ChatGLMConfig, layer_number, device=None):
        super(GLMBlock, self).__init__()
        self.layer_number = layer_number

        self.apply_residual_connection_post_layernorm = config.apply_residual_connection_post_layernorm
        self.fp32_residual_connection = config.fp32_residual_connection

        LayerNormFunc = RMSNorm if config.rmsnorm else LayerNorm
        # 输入数据上的层归一化
        self.input_layernorm = LayerNormFunc(config.hidden_size, eps=config.layernorm_epsilon, device=device,
                                             dtype=config.torch_dtype)

        # 自注意力层
        self.self_attention = SelfAttention(config, layer_number, device=device)
        self.hidden_dropout = config.hidden_dropout

        # 注意力输出上的层归一化
        self.post_attention_layernorm = LayerNormFunc(config.hidden_size, eps=config.layernorm_epsilon, device=device,
                                                      dtype=config.torch_dtype)

        # 多层感知机（MLP）
        self.mlp = MLP(config, device=device)

    def forward(
            self, hidden_states, attention_mask, rotary_pos_emb, kv_cache=None, use_cache=True,
    ):
        # hidden_states: [s, b, h]

        # 在 transformer 层开始时的层归一化
        layernorm_output = self.input_layernorm(hidden_states)
        # 自注意力层
        attention_output, kv_cache = self.self_attention(
            layernorm_output,
            attention_mask,
            rotary_pos_emb,
            kv_cache=kv_cache,
            use_cache=use_cache
        )

        # 残差连接
        if self.apply_residual_connection_post_layernorm:
            residual = layernorm_output
        else:
            residual = hidden_states

        layernorm_input = torch.nn.functional.dropout(attention_output, p=self.hidden_dropout, training=self.training)
        layernorm_input = residual + layernorm_input

        # 自注意力后的层归一化
        layernorm_output = self.post_attention_layernorm(layernorm_input)

        # 多层感知机（MLP）
        mlp_output = self.mlp(layernorm_output)

        # 第二个残差连接
        if self.apply_residual_connection_post_layernorm:
            residual = layernorm_output
        else:
            residual = layernorm_input

        output = torch.nn.functional.dropout(mlp_output, p=self.hidden_dropout, training=self.training)
        output = residual + output

        return output, kv_cache


class GLMTransformer(torch.nn.Module):
    """Transformer 类。"""

    def __init__(self, config: ChatGLMConfig, device=None):
        super(GLMTransformer, self).__init__()

        self.fp32_residual_connection = config.fp32_residual_connection
        self.post_layer_norm = config.post_layer_norm

        # 层数
        self.num_layers = config.num_layers

        # Transformer 层
        def build_layer(layer_number):
            return GLMBlock(config, layer_number, device=device)

        self.layers = torch.nn.ModuleList([build_layer(i + 1) for i in range(self.num_layers)])

        if self.post_layer_norm:
            LayerNormFunc = RMSNorm if config.rmsnorm else LayerNorm
            # 输出前的最终层归一化
            self.final_layernorm = LayerNormFunc(config.hidden_size, eps=config.layernorm_epsilon, device=device,
                                                 dtype=config.torch_dtype)

        self.gradient_checkpointing = False

    def _get_layer(self, layer_number):
        return self.layers[layer_number]

    def forward(
            self, hidden_states, attention_mask, rotary_pos_emb, kv_caches=None,
            use_cache: Optional[bool] = True,
            output_hidden_states: Optional[bool] = False,
    ):
        if not kv_caches:
            kv_caches = [None for _ in range(self.num_layers)]
        presents = () if use_cache else None
        if self.gradient_checkpointing and self.training:
            if use_cache:
                logger.warning_once(
                    "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                )
                use_cache = False

        all_self_attentions = None
        all_hidden_states = () if output_hidden_states else None
        for index in range(self.num_layers):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer = self._get_layer(index)
            if self.gradient_checkpointing and self.training:
                layer_ret = torch.utils.checkpoint.checkpoint(
                    layer,
                    hidden_states,
                    attention_mask,
                    rotary_pos_emb,
                    kv_caches[index],
                    use_cache,
                    use_reentrant=False
                )
            else:
                layer_ret = layer(
                    hidden_states,
                    attention_mask,
                    rotary_pos_emb,
                    kv_cache=kv_caches[index],
                    use_cache=use_cache
                )
            hidden_states, kv_cache = layer_ret
            if use_cache:
                # token by token 解码，使用元组格式
                if kv_caches[0] is not None:
                    presents = presents + (kv_cache,)
                # 预填充解码，使用张量格式以节省 CUDA 内存
                else:
                    if len(presents) == 0:
                        presents = kv_cache
                    else:
                        presents = torch.cat((presents, kv_cache.to(presents.device)), dim=0)

        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_states,)

        # 最终层归一化
        if self.post_layer_norm:
            hidden_states = self.final_layernorm(hidden_states)

        return hidden_states, presents, all_hidden_states, all_self_attentions


## 张量形状变化及其意义。

### `CoreAttention` 类中的张量形状

1. **`query_layer`, `key_layer`, `value_layer` 形状变化**:
   ```python
   # 输入形状 [b, np, sq, hn] 和 [b, np, sk, hn]
   # [b, np, sq, hn] -> [b * np, sq, hn]
   query_layer = query_layer.view(output_size[0] * output_size[1], output_size[2], -1)
   # [b, np, sk, hn] -> [b * np, sk, hn]
   key_layer = key_layer.view(output_size[0] * output_size[1], output_size[3], -1)
   ```

2. **原始注意力得分的计算**:
   ```python
   # 输入形状 [b * np, sq, hn] 和 [b * np, hn, sk]
   # 结果形状 [b * np, sq, sk]
   matmul_result = torch.baddbmm(matmul_input_buffer, query_layer, key_layer.transpose(1, 2), beta=0.0, alpha=(1.0 / self.norm_factor))
   ```

3. **调整视图到原始形状**:
   ```python
   # [b * np, sq, sk] -> [b, np, sq, sk]
   attention_scores = matmul_result.view(*output_size)
   ```

4. **注意力概率形状变化**:
   ```python
   # 输入形状 [b, np, sq, sk]
   # 调整后的形状 [b * np, sq, sk]
   attention_probs = attention_probs.view(output_size[0] * output_size[1], output_size[2], -1)
   ```

5. **计算上下文层**:
   ```python
   # 输入形状 [b * np, sq, sk] 和 [b * np, sk, hn]
   # 结果形状 [b * np, sq, hn]
   context_layer = torch.bmm(attention_probs, value_layer)
   # 调整视图 [b * np, sq, hn] -> [b, np, sq, hn]
   context_layer = context_layer.view(*output_size)
   # [b, np, sq, hn] -> [b, sq, np, hn]
   context_layer = context_layer.transpose(1, 2).contiguous()
   # [b, sq, np, hn] -> [b, sq, hp]
   new_context_layer_shape = context_layer.size()[:-2] + (self.hidden_size_per_partition,)
   context_layer = context_layer.reshape(*new_context_layer_shape)
   ```

### `SelfAttention` 类中的张量形状

1. **`mixed_x_layer` 形状变化**:
   ```python
   # 输入形状 [b, sq, h]
   # 结果形状 [b, sq, (np * 3 * hn)]
   mixed_x_layer = self.query_key_value(hidden_states)
   ```

2. **多查询注意力的形状变化**:
   ```python
   # [b, sq, (np * 3 * hn)] -> [b, sq, np, hn] 和 [b, sq, np, hn]
   (query_layer, key_layer, value_layer) = mixed_x_layer.split([...], dim=-1)
   # 调整视图
   query_layer = query_layer.view(query_layer.size()[:-1] + (self.num_attention_heads_per_partition, self.hidden_size_per_attention_head))
   key_layer = key_layer.view(key_layer.size()[:-1] + (self.num_multi_query_groups_per_partition, self.hidden_size_per_attention_head))
   value_layer = value_layer.view(value_layer.size()[:-1] + (self.num_multi_query_groups_per_partition, self.hidden_size_per_attention_head))
   ```

3. **普通注意力的形状变化**:
   ```python
   # [b, sq, (np * 3 * hn)] -> [b, sq, np, 3 * hn]
   new_tensor_shape = mixed_x_layer.size()[:-1] + (self.num_attention_heads_per_partition, 3 * self.hidden_size_per_attention_head)
   mixed_x_layer = mixed_x_layer.view(*new_tensor_shape)
   # [b, sq, np, 3 * hn] -> 3 [b, sq, np, hn]
   (query_layer, key_layer, value_layer) = split_tensor_along_last_dim(mixed_x_layer, 3)
   ```

4. **转置操作**:
   ```python
   # [b, sq, np, hn] -> [b, np, sq, hn]
   query_layer, key_layer, value_layer = [k.transpose(1, 2) for k in [query_layer, key_layer, value_layer]]
   ```

### `MLP` 类中的张量形状

1. **前向传播**:
   ```python
   # 输入形状 [s, b, h]
   # 经过 dense_h_to_4h 线性层后形状 [s, b, 4hp]
   intermediate_parallel = self.dense_h_to_4h(hidden_states)
   intermediate_parallel = self.activation_func(intermediate_parallel)
   # 经过 dense_4h_to_h 线性层后形状 [s, b, h]
   output = self.dense_4h_to_h(intermediate_parallel)
   ```

### `GLMBlock` 类中的张量形状

1. **自注意力层的输入输出形状**:
   ```python
   # 输入形状 [s, b, h]
   # 自注意力层输出形状 [s, b, h]
   attention_output, kv_cache = self.self_attention(layernorm_output, attention_mask, rotary_pos_emb, kv_cache=kv_cache, use_cache=use_cache)
   ```

2. **残差连接**:
   ```python
   # 残差连接，形状保持不变 [s, b, h]
   layernorm_input = residual + layernorm_input
   ```

3. **多层感知机（MLP）的输入输出形状**:
   ```python
   # MLP 输出形状 [s, b, h]
   mlp_output = self.mlp(layernorm_output)
   ```

### `GLMTransformer` 类中的张量形状

1. **逐层处理**:
   ```python
   # 每层的输入输出形状 [s, b, h]
   for index in range(self.num_layers):
       layer = self._get_layer(index)
       hidden_states, kv_cache = layer(hidden_states, attention_mask, rotary_pos_emb, kv_cache=kv_caches[index], use_cache=use_cache)
   ```

2. **最终层归一化**:
   ```python
   # 最终层归一化，形状 [s, b, h]
   if self.post_layer_norm:
       hidden_states = self.final_layernorm(hidden_states)
   ```

总结这些形状变化，有助于理解每个层的输入输出如何传递和处理，确保模型在每个步骤中保持正确的张量形状。

### 关键点解释

1. **`CoreAttention` 类**：
   - 实现了核心注意力机制。
   - 支持 PyTorch 2.0 及以上版本的 `scaled_dot_product_attention`，以及旧版本的自定义注意力计算。

2. **`SelfAttention` 类**：
   - 实现了自注意力层。
   - 包括 query、key、value 的计算以及核心注意力机制的应用。
   - 支持多查询注意力机制。

3. **`MLP` 类**：
   - 多层感知机（MLP），包括两个线性层和一个非线性激活函数。

4. **`GLMBlock` 类**：
   - 实现了一个 Transformer 层，包括自注意力层和 MLP 层。
   - 包括层归一化和残差连接。

5. **`GLMTransformer` 类**：
   - 实现了 Transformer 模型，包括多个 Transformer 层。
   - 支持梯度检查点和最终层归一化。

### 自注意力机制提取输入隐藏状态的步骤

在 SelfAttention 类中，自注意力机制通过一系列的步骤来提取和处理输入的隐藏状态（hidden states），最终生成上下文向量。这些步骤包括线性变换、计算注意力得分、应用注意力掩码、计算注意力权重和生成上下文向量。以下是详细的步骤和解释：

#### 1. 输入隐藏状态

输入隐藏状态 \( \text{hidden_states} \) 的形状通常为 \([b, sq, h]\)，其中：
- \( b \) 是批次大小（batch size）。
- \( sq \) 是序列长度（sequence length）。
- \( h \) 是隐藏层维度（hidden size）。

#### 2. 线性变换

将输入隐藏状态 \( \text{hidden_states} \) 通过线性层投影到查询 \( Q \)、键 \( K \) 和值 \( V \) 空间。这一步的目的是将输入映射到不同的子空间，以便进行注意力计算。公式如下：
\begin{align*} Q = \text{hidden_states} \cdot W_Q \end{align*}
\begin{align*} K = \text{hidden_states} \cdot W_K \end{align*}
\begin{align*} V = \text{hidden_states} \cdot W_V \end{align*}

在代码中，通过以下方式实现：
```python
mixed_x_layer = self.query_key_value(hidden_states)
```
这里 `self.query_key_value` 是一个线性层，它将输入隐藏状态投影到一个更大的空间，结果的形状为 \([b, sq, (3 \times \text{hidden_size})]\)。

#### 3. 拆分查询、键和值

将上述结果拆分成查询、键和值向量。拆分后，每个向量的形状为 \([b, sq, \text{hidden_size}]\)。
```python
(query_layer, key_layer, value_layer) = split_tensor_along_last_dim(mixed_x_layer, 3)
```
`split_tensor_along_last_dim` 函数按最后一个维度将张量分割成三部分。

#### 4. 形状变换

为了适应后续的矩阵乘法操作，需要调整查询、键和值的形状。将它们从 \([b, sq, np, hn]\) 转换为 \([b, np, sq, hn]\)，其中 \( np \) 是注意力头的数量，\( hn \) 是每个注意力头的维度。
```python
query_layer, key_layer, value_layer = [k.transpose(1, 2) for k in [query_layer, key_layer, value_layer]]
```

#### 5. 应用旋转位置编码

如果存在旋转位置编码（rotary position embedding），则应用到查询和键向量上。旋转位置编码可以帮助模型更好地捕捉位置信息。
```python
if rotary_pos_emb is not None:
    query_layer = apply_rotary_pos_emb(query_layer, rotary_pos_emb)
    key_layer = apply_rotary_pos_emb(key_layer, rotary_pos_emb)
```

#### 6. 调整键和值用于推理

在推理阶段，可能需要缓存键和值，以便在下一个时间步中重复使用，从而提高效率。将缓存的键和值与当前时间步的键和值进行拼接。
```python
if kv_cache is not None:
    cache_k, cache_v = kv_cache
    key_layer = torch.cat((cache_k, key_layer), dim=2)
    value_layer = torch.cat((cache_v, value_layer), dim=2)
if use_cache:
    if kv_cache is None:
        kv_cache = torch.cat((key_layer.unsqueeze(0).unsqueeze(0), value_layer.unsqueeze(0).unsqueeze(0)), dim=1)
    else:
        kv_cache = (key_layer, value_layer)
else:
    kv_cache = None
```

#### 7. 核心注意力计算

使用 CoreAttention 类来计算注意力得分、注意力权重和上下文向量。
```python
context_layer = self.core_attention(query_layer, key_layer, value_layer, attention_mask)
```

#### 8. 输出层

最后，将上下文向量通过线性层投影回原始的隐藏层维度。这一步是将计算后的结果映射回输入的形状，以便进行后续处理。
```python
output = self.dense(context_layer)
```

### 总结

通过上述步骤，自注意力机制实现了从输入隐藏状态中提取和处理信息的过程。每一步的详细解释如下：

1. **输入隐藏状态**：输入序列的隐藏表示。
2. **线性变换**：将隐藏表示投影到查询、键和值的空间。
3. **拆分查询、键和值**：将投影后的结果拆分成查询、键和值向量。
4. **形状变换**：调整查询、键和值的形状，以适应后续操作。
5. **应用旋转位置编码**：增强查询和键向量的位置信息。
6. **调整键和值用于推理**：在推理阶段缓存键和值，以提高效率。
7. **核心注意力计算**：通过计算注意力得分、权重和上下文向量，完成注意力机制的核心部分。
8. **输出层**：将上下文向量投影回原始的隐藏层维度。

这些步骤共同构成了自注意力机制，从而使模型能够动态地关注输入序列中的不同部分，捕捉全局信息并生成更丰富的表示。

### ChatGLMPreTrainedModel 类

In [70]:
class ChatGLMPreTrainedModel(PreTrainedModel):
    """
    处理权重初始化和下载及加载预训练模型的简单接口的抽象类。
    """
    is_parallelizable = False  # 是否可并行化
    supports_gradient_checkpointing = True  # 是否支持梯度检查点
    config_class = ChatGLMConfig  # 配置类
    base_model_prefix = "transformer"  # 基础模型前缀
    _no_split_modules = ["GLMBlock"]  # 不拆分的模块列表

    def _init_weights(self, module: nn.Module):
        """初始化权重"""
        return

    def get_masks(self, input_ids, past_key_values, padding_mask=None):
        """
        获取注意力掩码
        """
        batch_size, seq_length = input_ids.shape
        # 创建下三角矩阵作为全注意力掩码
        full_attention_mask = torch.ones(batch_size, seq_length, seq_length, device=input_ids.device)
        full_attention_mask.tril_()
        past_length = 0
        if past_key_values:
            past_length = past_key_values[0][0].shape[2]
        if past_length:
            full_attention_mask = torch.cat((torch.ones(batch_size, seq_length, past_length, device=input_ids.device), full_attention_mask), dim=-1)
        if padding_mask is not None:
            full_attention_mask = full_attention_mask * padding_mask.unsqueeze(1)
        if not past_length and padding_mask is not None:
            full_attention_mask -= padding_mask.unsqueeze(-1) - 1
        full_attention_mask = (full_attention_mask < 0.5).bool()
        full_attention_mask.unsqueeze_(1)
        return full_attention_mask

    def get_position_ids(self, input_ids, device):
        """
        获取位置ID
        """
        batch_size, seq_length = input_ids.shape
        position_ids = torch.arange(seq_length, dtype=torch.long, device=device).unsqueeze(0).repeat(batch_size, 1)
        return position_ids

    def gradient_checkpointing_enable(self, gradient_checkpointing_kwargs=None):
        if not self.supports_gradient_checkpointing:
            raise ValueError(f"{self.__class__.__name__} does not support gradient checkpointing.")

### Embedding 类

In [71]:
class Embedding(torch.nn.Module):
    """语言模型嵌入层。"""

    def __init__(self, config: ChatGLMConfig, device=None):
        super(Embedding, self).__init__()

        self.hidden_size = config.hidden_size
        # 词嵌入层（并行）
        self.word_embeddings = nn.Embedding(
            config.padded_vocab_size,
            self.hidden_size,
            dtype=config.torch_dtype,
            device=device
        )
        self.fp32_residual_connection = config.fp32_residual_connection

    def forward(self, input_ids):
        # 获取词嵌入
        words_embeddings = self.word_embeddings(input_ids)
        embeddings = words_embeddings
        # 如果设置了fp32残差连接，则转换为浮点数
        if self.fp32_residual_connection:
            embeddings = embeddings.float()
        return embeddings

### ChatGLMModel 类

In [72]:
class ChatGLMModel(ChatGLMPreTrainedModel):
    def __init__(self, config: ChatGLMConfig, device=None, empty_init=True):
        super().__init__(config)
        if empty_init:
            init_method = skip_init  # 跳过初始化
        else:
            init_method = default_init  # 使用默认初始化方法
        init_kwargs = {}
        if device is not None:
            init_kwargs["device"] = device
        self.embedding = init_method(Embedding, config, **init_kwargs)  # 使用 Embedding 类
        self.num_layers = config.num_layers
        self.multi_query_group_num = config.multi_query_group_num
        self.kv_channels = config.kv_channels

        # 旋转位置嵌入
        self.seq_length = config.seq_length
        rotary_dim = (
            config.hidden_size // config.num_attention_heads if config.kv_channels is None else config.kv_channels
        )

        self.rotary_pos_emb = RotaryEmbedding(rotary_dim // 2, rope_ratio=config.rope_ratio, original_impl=config.original_rope, 
                                              device=device, dtype=config.torch_dtype)  # 使用 RotaryEmbedding 类
        self.encoder = init_method(GLMTransformer, config, **init_kwargs)  # 使用 GLMTransformer 类
        self.output_layer = init_method(nn.Linear, config.hidden_size, config.padded_vocab_size, bias=False,
                                        dtype=config.torch_dtype, **init_kwargs)  # 使用 nn.Linear 类

    def get_input_embeddings(self):
        return self.embedding.word_embeddings

    def set_input_embeddings(self, value):
        self.embedding.word_embeddings = value

    def forward(
            self,
            input_ids,
            position_ids: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.BoolTensor] = None,
            full_attention_mask: Optional[torch.BoolTensor] = None,
            past_key_values: Optional[Tuple[Tuple[torch.Tensor, torch.Tensor], ...]] = None,
            inputs_embeds: Optional[torch.Tensor] = None,
            use_cache: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
    ):
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        batch_size, seq_length = input_ids.shape

        if inputs_embeds is None:
            inputs_embeds = self.embedding(input_ids)  # 使用 Embedding 类

        if full_attention_mask is None:
            if (attention_mask is not None and not attention_mask.all()) or (past_key_values and seq_length != 1):
                full_attention_mask = self.get_masks(input_ids, past_key_values, padding_mask=attention_mask)

        # 旋转位置嵌入
        rotary_pos_emb = self.rotary_pos_emb(self.seq_length)  # 使用 RotaryEmbedding 类
        if position_ids is not None:
            rotary_pos_emb = rotary_pos_emb[position_ids]
        else:
            rotary_pos_emb = rotary_pos_emb[None, :seq_length]

        # 运行编码器
        hidden_states, presents, all_hidden_states, all_self_attentions = self.encoder(
            inputs_embeds, full_attention_mask, rotary_pos_emb=rotary_pos_emb,
            kv_caches=past_key_values, use_cache=use_cache, output_hidden_states=output_hidden_states
        )  # 使用 GLMTransformer 类
        if presents is not None and type(presents) is torch.Tensor:
            presents = presents.split(1, dim=0)
            presents = list(presents)
            presents = [list(x.squeeze(0).split(1, dim=0)) for x in presents]
            presents = [tuple([x.squeeze(0) for x in y]) for y in presents]
            presents = tuple(presents)

        if not return_dict:
            return tuple(v for v in [hidden_states, presents, all_hidden_states, all_self_attentions] if v is not None)

        return BaseModelOutputWithPast(
            last_hidden_state=hidden_states,
            past_key_values=presents,
            hidden_states=all_hidden_states,
            attentions=all_self_attentions,
        )

### 总结

1. **ChatGLMPreTrainedModel**:
   - **get_masks**：生成注意力掩码。
   - **get_position_ids**：生成位置 ID。

2. **Embedding**:
   - **word_embeddings**：实现词嵌入。

3. **ChatGLMModel**:
   - **RotaryEmbedding**：用于位置编码。
   - **GLMTransformer**：实现 Transformer 编码器。
   - **forward**：执行前向传播，集成所有模块。

4. **使用的先前构建的模块**：
   - **RotaryEmbedding**：用于生成旋转位置嵌入。
   - **GLMTransformer**：用于编码器部分。
   - **Embedding**：用于生成词嵌入。
   - **CoreAttention**、**SelfAttention**：间接通过 GLMTransformer 使用。

### ChatGLMForConditionalGeneration 类

In [73]:
class ChatGLMForConditionalGeneration(ChatGLMPreTrainedModel):
    def __init__(self, config: ChatGLMConfig, empty_init=True, device=None):
        super().__init__(config)

        self.max_sequence_length = config.max_length  # 最大序列长度
        self.transformer = ChatGLMModel(config, empty_init=empty_init, device=device)  # 使用 ChatGLMModel 类
        self.config = config

    def _update_model_kwargs_for_generation(
            self,
            outputs: ModelOutput,
            model_kwargs: Dict[str, Any],
            is_encoder_decoder: bool = False,
            standardize_cache_format: bool = False,
    ) -> Dict[str, Any]:
        # 更新 past_key_values
        model_kwargs["past_key_values"] = self._extract_past_from_model_output(
            outputs, standardize_cache_format=standardize_cache_format
        )

        # 更新注意力掩码
        if "attention_mask" in model_kwargs:
            attention_mask = model_kwargs["attention_mask"]
            model_kwargs["attention_mask"] = torch.cat(
                [attention_mask, attention_mask.new_ones((attention_mask.shape[0], 1))], dim=-1
            )

        # 更新位置 ids
        if "position_ids" in model_kwargs:
            position_ids = model_kwargs["position_ids"]
            new_position_id = position_ids[..., -1:].clone()
            new_position_id += 1
            model_kwargs["position_ids"] = torch.cat(
                [position_ids, new_position_id], dim=-1
            )

        model_kwargs["is_first_forward"] = False
        return model_kwargs

    def prepare_inputs_for_generation(
            self,
            input_ids: torch.LongTensor,
            past_key_values: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.Tensor] = None,
            use_cache: Optional[bool] = None,
            is_first_forward: bool = True,
            **kwargs
    ) -> dict:
        # 如果 past_key_values 不为空，只取 input_ids 的最后一个 token
        if position_ids is None:
            position_ids = self.get_position_ids(input_ids, device=input_ids.device)
        if not is_first_forward:
            if past_key_values is not None:
                position_ids = position_ids[..., -1:]
                input_ids = input_ids[:, -1:]
        return {
            "input_ids": input_ids,
            "past_key_values": past_key_values,
            "position_ids": position_ids,
            "attention_mask": attention_mask,
            "return_last_logit": True,
            "use_cache": use_cache
        }

    def forward(
            self,
            input_ids: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.Tensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            past_key_values: Optional[Tuple[torch.FloatTensor]] = None,
            inputs_embeds: Optional[torch.Tensor] = None,
            labels: Optional[torch.Tensor] = None,
            use_cache: Optional[bool] = None,
            output_attentions: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
            return_last_logit: Optional[bool] = False,
    ):
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.transformer(
            input_ids=input_ids,
            position_ids=position_ids,
            attention_mask=attention_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )  # 使用 ChatGLMModel 类

        hidden_states = transformer_outputs[0]
        if return_last_logit:
            hidden_states = hidden_states[:, -1:]
        lm_logits = self.transformer.output_layer(hidden_states)

        loss = None
        if labels is not None:
            lm_logits = lm_logits.to(torch.float32)

            # Shift so that tokens < n predict n
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            lm_logits = lm_logits.to(hidden_states.dtype)
            loss = loss.to(hidden_states.dtype)

        if not return_dict:
            output = (lm_logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return CausalLMOutputWithPast(
            loss=loss,
            logits=lm_logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )

    @staticmethod
    def _reorder_cache(
            past: Tuple[Tuple[torch.Tensor, torch.Tensor], ...], beam_idx: torch.LongTensor
    ) -> Tuple[Tuple[torch.Tensor, torch.Tensor], ...]:
        """
        重新排序 `past_key_values` 缓存以匹配每个生成步骤中的 `beam_idx`。
        """
        return tuple(
            (
                layer_past[0].index_select(0, beam_idx.to(layer_past[0].device)),
                layer_past[1].index_select(0, beam_idx.to(layer_past[1].device)),
            )
            for layer_past in past
        )

    def process_response(self, output, history):
        content = ""
        history = deepcopy(history)
        for response in output.split(""):
            if "\n" in response:
                metadata, content = response.split("\n", maxsplit=1)
            else:
                metadata, content = "", response
            if not metadata.strip():
                content = content.strip()
                history.append({"role": "assistant", "metadata": metadata, "content": content})
                content = content.replace("[[训练时间]]", "2023年")
            else:
                history.append({"role": "assistant", "metadata": metadata, "content": content})
                if history[0]["role"] == "system" and "tools" in history[0]:
                    parameters = json.loads(content)
                    content = {"name": metadata.strip(), "parameters": parameters}
                else:
                    content = {"name": metadata.strip(), "content": content}
        return content, history

    @torch.inference_mode()
    def chat(self, tokenizer, query: str, history: List[Dict] = None, role: str = "user",
             max_length: int = 8192, num_beams=1, do_sample=True, top_p=0.8, temperature=0.8, logits_processor=None,
             **kwargs):
        if history is None:
            history = []
        if logits_processor is None:
            logits_processor = LogitsProcessorList()
        logits_processor.append(InvalidScoreLogitsProcessor())
        gen_kwargs = {"max_length": max_length, "num_beams": num_beams, "do_sample": do_sample, "top_p": top_p,
                      "temperature": temperature, "logits_processor": logits_processor, **kwargs}
        history.append({"role": role, "content": query})
        inputs = tokenizer.apply_chat_template(history, add_generation_prompt=True, tokenize=True,
                                               return_tensors="pt", return_dict=True)
        inputs = inputs.to(self.device)
        eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(""),
                        tokenizer.convert_tokens_to_ids("")]
        outputs = self.generate(**inputs, **gen_kwargs, eos_token_id=eos_token_id)
        outputs = outputs.tolist()[0][len(inputs["input_ids"][0]):-1]
        response = tokenizer.decode(outputs)
        response, history = self.process_response(response, history)
        return response, history

    @torch.inference_mode()
    def stream_chat(self, tokenizer, query: str, history: List[Dict] = None, role: str = "user",
                    past_key_values=None, max_length: int = 8192, do_sample=True, top_p=0.8, temperature=0.8,
                    logits_processor=None, return_past_key_values=False, **kwargs):
        if history is None:
            history = []
        if logits_processor is None:
            logits_processor = LogitsProcessorList()
        logits_processor.append(InvalidScoreLogitsProcessor())
        eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(""),
                        tokenizer.convert_tokens_to_ids("")]
        gen_kwargs = {"max_length": max_length, "do_sample": do_sample, "top_p": top_p,
                      "temperature": temperature, "logits_processor": logits_processor, **kwargs}
        if past_key_values is None:
            inputs = tokenizer.apply_chat_template(history + [{"role": role, "content": query}],
                                                   add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                                   return_dict=True)
        else:
            inputs = tokenizer.apply_chat_template([{"role": role, "content": query}], add_special_tokens=False,
                                                   add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                                   return_dict=True)
        inputs = inputs.to(self.device)
        if past_key_values is not None:
            past_length = past_key_values[0][0].shape[2]
            inputs.position_ids += past_length


            attention_mask = inputs.attention_mask
            attention_mask = torch.cat((attention_mask.new_ones(1, past_length), attention_mask), dim=1)
            inputs['attention_mask'] = attention_mask
        history.append({"role": role, "content": query})
        for outputs in self.stream_generate(**inputs, past_key_values=past_key_values,
                                            eos_token_id=eos_token_id, return_past_key_values=return_past_key_values,
                                            **gen_kwargs):
            if return_past_key_values:
                outputs, past_key_values = outputs
            outputs = outputs.tolist()[0][len(inputs["input_ids"][0]):-1]
            response = tokenizer.decode(outputs)
            if response and response[-1] != "�":
                response, new_history = self.process_response(response, history)
                if return_past_key_values:
                    yield response, new_history, past_key_values
                else:
                    yield response, new_history

    @torch.inference_mode()
    def stream_generate(
            self,
            input_ids,
            generation_config: Optional[GenerationConfig] = None,
            logits_processor: Optional[LogitsProcessorList] = None,
            stopping_criteria: Optional[StoppingCriteriaList] = None,
            prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
            return_past_key_values=False,
            **kwargs,
    ):
        batch_size, input_ids_seq_length = input_ids.shape[0], input_ids.shape[-1]

        if generation_config is None:
            generation_config = self.generation_config
        generation_config = copy.deepcopy(generation_config)
        model_kwargs = generation_config.update(**kwargs)
        model_kwargs["use_cache"] = generation_config.use_cache
        bos_token_id, eos_token_id = generation_config.bos_token_id, generation_config.eos_token_id

        if isinstance(eos_token_id, int):
            eos_token_id = [eos_token_id]
        eos_token_id_tensor = torch.tensor(eos_token_id).to(input_ids.device) if eos_token_id is not None else None

        has_default_max_length = kwargs.get("max_length") is None and generation_config.max_length is not None
        if has_default_max_length and generation_config.max_new_tokens is None:
            warnings.warn(
                f"Using `max_length`'s default ({generation_config.max_length}) to control the generation length. "
                "This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we"
                " recommend using `max_new_tokens` to control the maximum length of the generation.",
                UserWarning,
            )
        elif generation_config.max_new_tokens is not None:
            generation_config.max_length = generation_config.max_new_tokens + input_ids_seq_length
            if not has_default_max_length:
                logger.warn(
                    f"Both `max_new_tokens` (={generation_config.max_new_tokens}) and `max_length`(="
                    f"{generation_config.max_length}) seem to have been set. `max_new_tokens` will take precedence. "
                    "Please refer to the documentation for more information. "
                    "(https://hf-mirror.com/docs/transformers/main/en/main_classes/text_generation)",
                    UserWarning,
                )

        if input_ids_seq_length >= generation_config.max_length:
            input_ids_string = "decoder_input_ids" if self.config.is_encoder_decoder else "input_ids"
            logger.warning(
                f"Input length of {input_ids_string} is {input_ids_seq_length}, but `max_length` is set to"
                f" {generation_config.max_length}. This can lead to unexpected behavior. You should consider"
                " increasing `max_new_tokens`."
            )

        # 2. Set generation parameters if not already defined
        logits_processor = logits_processor if logits_processor is not None else LogitsProcessorList()
        stopping_criteria = stopping_criteria if stopping_criteria is not None else StoppingCriteriaList()

        logits_processor = self._get_logits_processor(
            generation_config=generation_config,
            input_ids_seq_length=input_ids_seq_length,
            encoder_input_ids=input_ids,
            prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
            logits_processor=logits_processor,
        )

        stopping_criteria = self._get_stopping_criteria(
            generation_config=generation_config, stopping_criteria=stopping_criteria
        )
        logits_warper = self._get_logits_warper(generation_config)

        unfinished_sequences = input_ids.new(input_ids.shape[0]).fill_(1)
        scores = None
        while True:
            model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
            # 前向传递获取下一个 token
            outputs = self(
                **model_inputs,
                return_dict=True,
                output_attentions=False,
                output_hidden_states=False,
            )

            next_token_logits = outputs.logits[:, -1, :]

            # 预处理分布
            next_token_scores = logits_processor(input_ids, next_token_logits)
            next_token_scores = logits_warper(input_ids, next_token_scores)

            # 采样
            probs = nn.functional.softmax(next_token_scores, dim=-1)
            if generation_config.do_sample:
                next_tokens = torch.multinomial(probs, num_samples=1).squeeze(1)
            else:
                next_tokens = torch.argmax(probs, dim=-1)
            # 更新生成的 ids、模型输入和下一个步骤的长度
            input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
            model_kwargs = self._update_model_kwargs_for_generation(
                outputs, model_kwargs, is_encoder_decoder=self.config.is_encoder_decoder
            )
            unfinished_sequences = unfinished_sequences.mul(
                next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)
            )
            if return_past_key_values:
                yield input_ids, outputs.past_key_values
            else:
                yield input_ids
            # 当每个句子完成时或超出最大长度时停止
            if unfinished_sequences.max() == 0 or stopping_criteria(input_ids, scores):
                break

### ChatGLMForSequenceClassification 类

In [74]:
class ChatGLMForSequenceClassification(ChatGLMPreTrainedModel):
    def __init__(self, config: ChatGLMConfig, empty_init=True, device=None):
        super().__init__(config)

        self.num_labels = config.num_labels  # 标签数量
        self.transformer = ChatGLMModel(config, empty_init=empty_init, device=device)  # 使用 ChatGLMModel 类

        self.classifier_head = nn.Linear(config.hidden_size, config.num_labels, bias=True, dtype=torch.half)
        if config.classifier_dropout is not None:
            self.dropout = nn.Dropout(config.classifier_dropout)
        else:
            self.dropout = None
        self.config = config

    def forward(
            self,
            input_ids: Optional[torch.LongTensor] = None,
            position_ids: Optional[torch.LongTensor] = None,
            attention_mask: Optional[torch.Tensor] = None,
            full_attention_mask: Optional[torch.Tensor] = None,
            past_key_values: Optional[Tuple[Tuple[torch.Tensor, torch.Tensor], ...]] = None,
            inputs_embeds: Optional[torch.LongTensor] = None,
            labels: Optional[torch.LongTensor] = None,
            use_cache: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor, ...], SequenceClassifierOutputWithPast]:
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.transformer(
            input_ids=input_ids,
            position_ids=position_ids,
            attention_mask=attention_mask,
            full_attention_mask=full_attention_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )  # 使用 ChatGLMModel 类

        hidden_states = transformer_outputs[0]
        pooled_hidden_states = hidden_states[:, -1]
        if self.dropout is not None:
            pooled_hidden_states = self.dropout(pooled_hidden_states)
        logits = self.classifier_head(pooled_hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze().float(), labels.squeeze())
                else:
                    loss = loss_fct(logits.float(), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels).float(), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits.float(), labels.view(-1, self.num_labels))

        if not return_dict:
            output = (logits,) + transformer_outputs[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutputWithPast(
            loss

=loss,
            logits=logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )


### 使用的先前构建的模块

在这段代码中，多个模块和方法是基于之前构建的类和函数的：

1. **ChatGLMModel**：
   - **用于 ChatGLMForConditionalGeneration 和 ChatGLMForSequenceClassification 中**，作为 Transformer 模型的核心部分。

2. **ChatGLMPreTrainedModel**：
   - **作为 ChatGLMForConditionalGeneration 和 ChatGLMForSequenceClassification 的基类**，提供权重初始化和加载预训练模型的接口。

3. **RotaryEmbedding**：
   - **在 ChatGLMModel 中用于位置编码**。

4. **CoreAttention 和 SelfAttention**：
   - **在 GLMTransformer 中使用**，实现了注意力机制的核心部分。

通过详细注释和说明，可以更好地理解代码的构建和实现原理。这些模块共同构成了 ChatGLM 模型的整体架构，实现了条件生成和序列分类的功能。

### `past_key_values` 变量的含义

在 Transformer 模型中，特别是用于生成任务的模型，如 GPT 类模型中，`past_key_values` 是一个非常重要的变量。它用于缓存模型在前一个时间步计算得到的键（key）和值（value）向量。这些缓存的数据可以在后续的时间步中重复使用，从而提高计算效率，尤其是在长序列生成任务中。

### `past_key_values` 的作用

1. **缓存先前计算结果**：
   在生成文本的过程中，每一步生成一个新的词，这时需要将当前时间步的查询向量（query）与所有先前时间步的键和值向量进行计算。如果每次都重新计算所有的键和值，将会非常低效。`past_key_values` 缓存了这些先前时间步的结果，避免了重复计算。

2. **加速生成过程**：
   在长序列生成中，通过缓存先前时间步的键和值向量，只需要对当前时间步进行计算并与缓存结果结合，大大加速了生成过程。

### 具体实现中的 `past_key_values`

#### 在 `ChatGLMForConditionalGeneration` 类中的使用

```python
def prepare_inputs_for_generation(
        self,
        input_ids: torch.LongTensor,
        past_key_values: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        use_cache: Optional[bool] = None,
        is_first_forward: bool = True,
        **kwargs
) -> dict:
    # 如果 past_key_values 不为空，只取 input_ids 的最后一个 token
    if position_ids is None:
        position_ids = self.get_position_ids(input_ids, device=input_ids.device)
    if not is_first_forward:
        if past_key_values is not None:
            position_ids = position_ids[..., -1:]
            input_ids = input_ids[:, -1:]
    return {
        "input_ids": input_ids,
        "past_key_values": past_key_values,
        "position_ids": position_ids,
        "attention_mask": attention_mask,
        "return_last_logit": True,
        "use_cache": use_cache
    }
```

在 `prepare_inputs_for_generation` 方法中，如果 `past_key_values` 不为空，只会取 `input_ids` 的最后一个 token。这样做的目的是为了在生成新 token 时，只计算当前时间步的数据，而不需要重新计算整个序列。

#### 在 `ChatGLMForConditionalGeneration` 类的 `forward` 方法中

```python
def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[Tuple[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        return_last_logit: Optional[bool] = False,
):
    use_cache = use_cache if use_cache is not None else self.config.use_cache
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    transformer_outputs = self.transformer(
        input_ids=input_ids,
        position_ids=position_ids,
        attention_mask=attention_mask,
        past_key_values=past_key_values,
        inputs_embeds=inputs_embeds,
        use_cache=use_cache,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )  # 使用 ChatGLMModel 类

    hidden_states = transformer_outputs[0]
    if return_last_logit:
        hidden_states = hidden_states[:, -1:]
    lm_logits = self.transformer.output_layer(hidden_states)

    loss = None
    if labels is not None:
        lm_logits = lm_logits.to(torch.float32)

        # Shift so that tokens < n predict n
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        # Flatten the tokens
        loss_fct = CrossEntropyLoss(ignore_index=-100)
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        lm_logits = lm_logits.to(hidden_states.dtype)
        loss = loss.to(hidden_states.dtype)

    if not return_dict:
        output = (lm_logits,) + transformer_outputs[1:]
        return ((loss,) + output) if loss is not None else output

    return CausalLMOutputWithPast(
        loss=loss,
        logits=lm_logits,
        past_key_values=transformer_outputs.past_key_values,
        hidden_states=transformer_outputs.hidden_states,
        attentions=transformer_outputs.attentions,
    )
```

在 `forward` 方法中，`past_key_values` 作为参数传递给 `transformer` 模型。`transformer` 模型内部会使用这些缓存的键和值向量来加速计算。
`past_key_values` 是一种用于加速 Transformer 模型在生成任务中的缓存机制。它保存了前一个时间步计算得到的键和值向量，避免了在每个时间步中重复计算这些向量，从而提高了生成过程的效率。通过使用 `past_key_values`，模型可以更快地生成长序列数据，这在实际应用中是非常重要的。

### 函数分析及其区别和联系

在 `ChatGLMForConditionalGeneration` 类中，有三个与生成有关的重要函数：`_update_model_kwargs_for_generation`、`prepare_inputs_for_generation` 和 `forward`。它们的作用、区别和联系如下：

#### 1. `_update_model_kwargs_for_generation` 函数

```python
def _update_model_kwargs_for_generation(
        self,
        outputs: ModelOutput,
        model_kwargs: Dict[str, Any],
        is_encoder_decoder: bool = False,
        standardize_cache_format: bool = False,
) -> Dict[str, Any]:
    # 更新 past_key_values
    model_kwargs["past_key_values"] = self._extract_past_from_model_output(
        outputs, standardize_cache_format=standardize_cache_format
    )

    # 更新注意力掩码
    if "attention_mask" in model_kwargs:
        attention_mask = model_kwargs["attention_mask"]
        model_kwargs["attention_mask"] = torch.cat(
            [attention_mask, attention_mask.new_ones((attention_mask.shape[0], 1))], dim=-1
        )

    # 更新位置 ids
    if "position_ids" in model_kwargs:
        position_ids = model_kwargs["position_ids"]
        new_position_id = position_ids[..., -1:].clone()
        new_position_id += 1
        model_kwargs["position_ids"] = torch.cat(
            [position_ids, new_position_id], dim=-1
        )

    model_kwargs["is_first_forward"] = False
    return model_kwargs
```

- **作用**：更新生成过程中所需的模型参数。具体包括：
  - 更新 `past_key_values` 以缓存先前计算的键和值向量。
  - 更新 `attention_mask` 以包括新的生成的 token。
  - 更新 `position_ids` 以增加新的位置 ID。
- **区别**：该函数不直接进行前向传播，而是更新模型参数，为下一步的生成做准备。
- **联系**：该函数在每一步生成新 token 后调用，用于更新模型参数，为下一步的生成做准备。

#### 2. `prepare_inputs_for_generation` 函数

```python
def prepare_inputs_for_generation(
        self,
        input_ids: torch.LongTensor,
        past_key_values: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        use_cache: Optional[bool] = None,
        is_first_forward: bool = True,
        **kwargs
) -> dict:
    # 如果 past_key_values 不为空，只取 input_ids 的最后一个 token
    if position_ids is None:
        position_ids = self.get_position_ids(input_ids, device=input_ids.device)
    if not is_first_forward:
        if past_key_values is not None:
            position_ids = position_ids[..., -1:]
            input_ids = input_ids[:, -1:]
    return {
        "input_ids": input_ids,
        "past_key_values": past_key_values,
        "position_ids": position_ids,
        "attention_mask": attention_mask,
        "return_last_logit": True,
        "use_cache": use_cache
    }
```

- **作用**：准备生成过程所需的输入。具体包括：
  - 获取或更新 `position_ids`。
  - 如果不是第一次前向传播，且存在 `past_key_values`，则只取 `input_ids` 和 `position_ids` 的最后一个 token。
- **区别**：该函数主要用于处理输入数据，确保输入数据的形状和内容适合当前生成步骤。
- **联系**：在每一步生成过程中，会调用该函数准备输入数据，尤其是处理 `past_key_values` 以提高生成效率。

#### 3. `forward` 函数

```python
def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        past_key_values: Optional[Tuple[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        return_last_logit: Optional[bool] = False,
):
    use_cache = use_cache if use_cache is not None else self.config.use_cache
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    transformer_outputs = self.transformer(
        input_ids=input_ids,
        position_ids=position_ids,
        attention_mask=attention_mask,
        past_key_values=past_key_values,
        inputs_embeds=inputs_embeds,
        use_cache=use_cache,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )  # 使用 ChatGLMModel 类

    hidden_states = transformer_outputs[0]
    if return_last_logit:
        hidden_states = hidden_states[:, -1:]
    lm_logits = self.transformer.output_layer(hidden_states)

    loss = None
    if labels is not None:
        lm_logits = lm_logits.to(torch.float32)

        # Shift so that tokens < n predict n
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        # Flatten the tokens
        loss_fct = CrossEntropyLoss(ignore_index=-100)
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        lm_logits = lm_logits.to(hidden_states.dtype)
        loss = loss.to(hidden_states.dtype)

    if not return_dict:
        output = (lm_logits,) + transformer_outputs[1:]
        return ((loss,) + output) if loss is not None else output

    return CausalLMOutputWithPast(
        loss=loss,
        logits=lm_logits,
        past_key_values=transformer_outputs.past_key_values,
        hidden_states=transformer_outputs.hidden_states,
        attentions=transformer_outputs.attentions,
    )
```

- **作用**：执行前向传播，生成模型的输出。具体包括：
  - 将输入数据传递给 `transformer`（`ChatGLMModel`），进行前向计算。
  - 计算语言模型的 logits 和（如果有标签）计算损失。
  - 返回模型输出，包括 logits、`past_key_values`、隐藏状态和注意力权重。
- **区别**：这是模型的核心前向传播逻辑，直接处理输入数据并生成输出。
- **联系**：`forward` 函数使用了 `ChatGLMModel` 类来进行实际的前向传播，并调用了之前的 `prepare_inputs_for_generation` 来准备输入。

### 联系和流程

1. **准备输入数据**：
   - `prepare_inputs_for_generation` 函数用于处理输入数据，尤其是处理 `past_key_values` 以便只传递必要的最后一个 token。
   
2. **执行前向传播**：
   - `forward` 函数使用准备好的输入数据进行前向传播，生成输出。

3. **更新模型参数**：
   - `_update_model_kwargs_for_generation` 函数在每一步生成之后，更新模型的关键参数（如 `past_key_values`、`attention_mask` 和 `position_ids`），确保在下一步生成中使用最新的数据。

通过这些函数的紧密配合，可以高效地实现生成任务中的前向传播和缓存管理，从而提高模型的生成效率和效果。

这三个函数是 `ChatGLMForConditionalGeneration` 类中的核心函数，分别用于处理不同的生成任务需求。它们之间有一定的联系，同时也有各自的用途和特点。以下是对它们的详细解释及其区别和联系：

### 1. `chat` 函数

```python
@torch.inference_mode()
def chat(self, tokenizer, query: str, history: List[Dict] = None, role: str = "user",
         max_length: int = 8192, num_beams=1, do_sample=True, top_p=0.8, temperature=0.8, logits_processor=None,
         **kwargs):
    if history is None:
        history = []
    if logits_processor is None:
        logits_processor = LogitsProcessorList()
    logits_processor.append(InvalidScoreLogitsProcessor())
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams, "do_sample": do_sample, "top_p": top_p,
                  "temperature": temperature, "logits_processor": logits_processor, **kwargs}
    history.append({"role": role, "content": query})
    inputs = tokenizer.apply_chat_template(history, add_generation_prompt=True, tokenize=True,
                                           return_tensors="pt", return_dict=True)
    inputs = inputs.to(self.device)
    eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(""), tokenizer.convert_tokens_to_ids("")]
    outputs = self.generate(**inputs, **gen_kwargs, eos_token_id=eos_token_id)
    outputs = outputs.tolist()[0][len(inputs["input_ids"][0]):-1]
    response = tokenizer.decode(outputs)
    response, history = self.process_response(response, history)
    return response, history
```

- **作用**：执行一次完整的聊天会话。将用户的查询和历史记录编码成模型输入，生成响应并更新历史记录。
- **区别**：这是一个高层次的接口，适用于一次性生成完整响应。适合用于需要立即获得完整回答的应用场景。
- **联系**：它依赖于 `generate` 函数来实际生成响应，并调用 `process_response` 函数来处理生成的输出。

### 2. `stream_chat` 函数

```python
@torch.inference_mode()
def stream_chat(self, tokenizer, query: str, history: List[Dict] = None, role: str = "user",
                past_key_values=None, max_length: int = 8192, do_sample=True, top_p=0.8, temperature=0.8,
                logits_processor=None, return_past_key_values=False, **kwargs):
    if history is None:
        history = []
    if logits_processor is None:
        logits_processor = LogitsProcessorList()
    logits_processor.append(InvalidScoreLogitsProcessor())
    eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(""), tokenizer.convert_tokens_to_ids("")]
    gen_kwargs = {"max_length": max_length, "do_sample": do_sample, "top_p": top_p,
                  "temperature": temperature, "logits_processor": logits_processor, **kwargs}
    if past_key_values is None:
        inputs = tokenizer.apply_chat_template(history + [{"role": role, "content": query}],
                                               add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                               return_dict=True)
    else:
        inputs = tokenizer.apply_chat_template([{"role": role, "content": query}], add_special_tokens=False,
                                               add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                               return_dict=True)
    inputs = inputs.to(self.device)
    if past_key_values is not None:
        past_length = past_key_values[0][0].shape[2]
        inputs.position_ids += past_length
        attention_mask = inputs.attention_mask
        attention_mask = torch.cat((attention_mask.new_ones(1, past_length), attention_mask), dim=1)
        inputs['attention_mask'] = attention_mask
    history.append({"role": role, "content": query})
    for outputs in self.stream_generate(**inputs, past_key_values=past_key_values,
                                        eos_token_id=eos_token_id, return_past_key_values=return_past_key_values,
                                        **gen_kwargs):
        if return_past_key_values:
            outputs, past_key_values = outputs
        outputs = outputs.tolist()[0][len(inputs["input_ids"][0]):-1]
        response = tokenizer.decode(outputs)
        if response and response[-1] != "�":
            response, new_history = self.process_response(response, history)
            if return_past_key_values:
                yield response, new_history, past_key_values
            else:
                yield response, new_history
```

- **作用**：实现流式聊天会话。与 `chat` 函数类似，但它通过生成器逐步返回响应，适用于流式生成应用场景。
- **区别**：支持逐步生成响应，使得可以在生成过程中动态处理和显示部分响应。
- **联系**：依赖于 `stream_generate` 函数来逐步生成响应，并在每次生成新的响应片段后调用 `process_response` 函数来处理和更新历史记录。

### 3. `stream_generate` 函数

```python
@torch.inference_mode()
def stream_generate(
        self,
        input_ids,
        generation_config: Optional[GenerationConfig] = None,
        logits_processor: Optional[LogitsProcessorList] = None,
        stopping_criteria: Optional[StoppingCriteriaList] = None,
        prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None,
        return_past_key_values=False,
        **kwargs,
):
    batch_size, input_ids_seq_length = input_ids.shape[0], input_ids.shape[-1]

    if generation_config is None:
        generation_config = self.generation_config
    generation_config = copy.deepcopy(generation_config)
    model_kwargs = generation_config.update(**kwargs)
    model_kwargs["use_cache"] = generation_config.use_cache
    bos_token_id, eos_token_id = generation_config.bos_token_id, generation_config.eos_token_id

    if isinstance(eos_token_id, int):
        eos_token_id = [eos_token_id]
    eos_token_id_tensor = torch.tensor(eos_token_id).to(input_ids.device) if eos_token_id is not None else None

    has_default_max_length = kwargs.get("max_length") is None and generation_config.max_length is not None
    if has_default_max_length and generation_config.max_new_tokens is None:
        warnings.warn(
            f"Using `max_length`'s default ({generation_config.max_length}) to control the generation length. "
            "This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we"
            " recommend using `max_new_tokens` to control the maximum length of the generation.",
            UserWarning,
        )
    elif generation_config.max_new_tokens is not None:
        generation_config.max_length = generation_config.max_new_tokens + input_ids_seq_length
        if not has_default_max_length:
            logger.warn(
                f"Both `max_new_tokens` (={generation_config.max_new_tokens}) and `max_length`(="
                f"{generation_config.max_length}) seem to have been set. `max_new_tokens` will take precedence. "
                "Please refer to the documentation for more information. "
                "(https://hf-mirror.com/docs/transformers/main/en/main_classes/text_generation)",
                UserWarning,
            )

    if input_ids_seq_length >= generation_config.max_length:
        input_ids_string = "decoder_input_ids" if self.config.is_encoder_decoder else "input_ids"
        logger.warning(
            f"Input length of {input_ids_string} is {input_ids_seq_length}, but `max_length` is set to"
            f" {generation_config.max_length}. This can lead to unexpected behavior. You should consider"
            " increasing `max_new_tokens`."
        )

    # 2. Set generation parameters if not already defined
    logits_processor = logits_processor if logits_processor is not None else LogitsProcessorList()
    stopping_criteria = stopping_criteria if stopping_criteria is not None else StoppingCriteriaList()

    logits_processor = self._get_logits_processor(
        generation_config=generation_config,
        input_ids_seq_length=input_ids_seq_length,
        encoder_input_ids=input_ids,
        prefix_allowed_tokens_fn=prefix_allowed_tokens_fn,
        logits_processor=logits_processor,
    )

    stopping_criteria = self._get_stopping_criteria(
        generation_config=generation_config, stopping_criteria=stopping_criteria
    )
    logits_warper = self._get_logits_warper(generation_config)

    unfinished_sequences = input_ids.new(input_ids.shape[0]).fill_(1)
    scores = None
    while True:
        model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
        # 前向传递获取下一个 token
        outputs = self(
            **model_inputs,
            return_dict=True,
            output_attentions=False,
            output_hidden_states=False,
        )

        next_token_logits = outputs.logits[:, -1, :]

        # 预处理分布
        next_token_scores = logits_processor(input_ids, next_token_logits)
        next_token_scores = logits_warper(input_ids, next_token_scores)

        # 采样
        probs = nn.functional.softmax(next_token_scores, dim=-1)
        if generation_config.do_sample:
            next_tokens = torch.multinomial(probs, num_samples=1).squeeze(1)
        else:
            next_tokens = torch.argmax(probs, dim=-1)
       

 # 更新生成的 ids、模型输入和下一个步骤的长度
        input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
        model_kwargs = self._update_model_kwargs_for_generation(
            outputs, model_kwargs, is_encoder_decoder=self.config.is_encoder_decoder
        )
        unfinished_sequences = unfinished_sequences.mul(
            next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)
        )
        if return_past_key_values:
            yield input_ids, outputs.past_key_values
        else:
            yield input_ids
        # 当每个句子完成时或超出最大长度时停止
        if unfinished_sequences.max() == 0 or stopping_criteria(input_ids, scores):
            break
```

- **作用**：流式生成模型输出，逐步返回生成的 token 以便实时处理。
- **区别**：该函数通过生成器实现流式生成，每生成一个 token 就返回一次结果，适用于需要逐步展示生成结果的应用场景。
- **联系**：`stream_chat` 函数依赖 `stream_generate` 来逐步生成响应，并在每次生成新的 token 后更新输入和模型参数。

### 区别和联系总结

1. **区别**：
   - `chat` 函数：用于一次性生成完整响应，适合需要立即获得完整回答的应用场景。
   - `stream_chat` 函数：用于流式生成响应，适合逐步展示生成结果的应用场景。
   - `stream_generate` 函数：实现流式生成的核心逻辑，通过生成器逐步返回生成的 token。

2. **联系**：
   - `chat` 和 `stream_chat` 都是高层次接口，用户通过这些接口与模型交互。
   - `chat` 函数调用 `generate` 函数实现一次性生成，而 `stream_chat` 函数调用 `stream_generate` 函数实现流式生成。
   - `stream_generate` 函数使用了 `prepare_inputs_for_generation` 来处理输入数据，并通过 `_update_model_kwargs_for_generation` 更新模型参数，确保在生成过程中使用最新的数据。

通过这三个函数的协调工作，模型能够实现高效且灵活的生成任务，满足不同应用场景的需求。

In [75]:
device = "cuda"

tokenizer = ChatGLM4Tokenizer.from_pretrained("THUDM/glm-4-9b-chat", trust_remote_code=True)

query = "你好"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [76]:
model = ChatGLMForConditionalGeneration.from_pretrained(
    "THUDM/glm-4-9b-chat",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    load_in_4bit=True,
    device_map='auto'
).eval()#.to(device)

gen_kwargs = {"max_length": 300, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]


你好👋！有什么可以帮助你的吗？


耗时16s

### 备注：其他测试草稿

In [ ]:
def apply_chat_template(
        self,
        conversation: Union[List[Dict[str, str]], List[List[Dict[str, str]]], "Conversation"],
        add_generation_prompt: bool = False,
        tokenize: bool = True,
        padding: bool = False,
        truncation: bool = False,
        max_length: Optional[int] = None,
        return_tensors: Optional[Union[str, TensorType]] = None,
        return_dict: bool = False,
        tokenizer_kwargs: Optional[Dict[str, Any]] = None,
        add_special_tokens: bool = True,
        **kwargs,
) -> Union[str, List[int], List[str], List[List[int]], BatchEncoding]:

    if return_dict and not tokenize:
        raise ValueError(
            "`return_dict=True` is incompatible with `tokenize=False`, because there is no dict "
            "of tokenizer outputs to return."
        )

    def handle_single_conversation(conversation):
        input_ids = self.get_prefix_tokens() if add_special_tokens else []
        input_message = "[gMASK]<sop>" if add_special_tokens else ""
        for item in conversation:
            if item.get("tools"):
                tools = item["tools"]
                content = "你是一个名为 GLM-4 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。"
                for tool in tools:
                    if tool["type"] == "function":
                        function = tool["function"]
                        content += f"\n\n## {function['name']}\n\n{json.dumps(function, ensure_ascii=False, indent=4)}"
                        content += "\n在调用上述函数时，请使用 Json 格式表示调用的参数。"
                    elif tool["type"] == "python":
                        content += "\n\n## python\n\n当你向 `python` 发送包含 Python 代码的消息时，该代码将会在一个有状态的 Jupyter notebook 环境中执行。\n`python` 返回代码执行的输出，或在执行 60 秒后返回超时。\n`/mnt/data` 将会持久化存储你的文件。在此会话中，`python` 无法访问互联网。不要使用 `python` 进行任何网络请求或者在线 API 调用，这些在线内容的访问将不会成功。"
                    elif tool["type"] == "simple_browser":
                        content += "\n\n## simple_browser\n\n你可以使用 `simple_browser` 工具。该工具支持以下函数：\n`search(query: str, recency_days: int)`：使用搜索引擎进行查询并显示结果，可以使用 `recency_days` 参数控制搜索内容的时效性。\n`mclick(ids: list[int])`：获取一系列指定 id 的页面内容。每次调用时，须选择3-10个页面。选择多个角度的页面，同时尽可能选择可信任的信息来源。考虑到部分页面是无法加载的，你也可以多打开一些可能有用的页面而不用担心内容过多。\n`open_url(url: str)`：打开指定的 URL。\n\n使用 `【{引用 id}†{引用文本}】` 来引用内容。\n\n操作步骤：1. 使用 `search` 来获得信息列表; 2. 使用 `mclick` 来获取指定 ID 页面的内容; 3. 根据获得的内容进行回复。在回复中应当引用信息来源。\n 如果用户提供了 URL，也可以用 `open_url` 直接打开页面。\n如果初次搜索结果没有找到合适的信息，也可以再次使用 `search` 进行搜索。"
                    elif tool["type"] == "cogview":
                        content += "\n\n## cogview\n\n如果用户的请求中包含了对图像的描述，你可以使用 `cogview` 来生成图像并展示给用户。你需要向 `cogview` 发送图像描述，规则：\n- 发送给 `cogview` 的消息必须使用英语。用户的中文描述必须完全翻译为英语。\n- 应当尽可能详细地描述图像生成的需求，需求描述约 100 英文单词。\n- 保持用户原始描述的意图。不要虚构内容或者没见过的人物。\n- 如无特殊说明，所在地为中国，持有中国立场并遵循中国社会主义价值观。"
                    else:
                        raise NotImplementedError(f"Unknown tool type {tool['type']}")
                input = self.build_single_message("system", "", content, tokenize=tokenize)
                if tokenize:
                    input_ids.extend(input)
                else:
                    input_message += input
            if item["content"]:
                input = self.build_single_message(
                    item["role"],
                    item.get("metadata", ""),
                    item["content"],
                    tokenize=tokenize
                )
                if tokenize:
                    input_ids.extend(input)
                else:
                    input_message += input
        if add_generation_prompt:
            if tokenize:
                input_ids.extend([self.convert_tokens_to_ids("<|assistant|>")])
            else:
                input_message += "<|assistant|>"

        return input_ids if tokenize else input_message

    # Main logic to handle different conversation formats
    if isinstance(conversation, list) and all(isinstance(i, dict) for i in conversation):
        result = handle_single_conversation(conversation)
    elif isinstance(conversation, list) and all(isinstance(i, list) for i in conversation):
        result = [handle_single_conversation(c) for c in conversation]
    elif hasattr(conversation, "messages"):
        result = handle_single_conversation(conversation.messages)
    else:
        raise ValueError("Invalid conversation format")

    if tokenize:
        output = self.batch_encode_plus(
            [result] if isinstance(result[0], int) else result,
            padding=padding,
            truncation=truncation,
            max_length=max_length,
            return_tensors=return_tensors,
            is_split_into_words=True,
            add_special_tokens=False
        )
        if return_dict:
            return output
        else:
            return output["input_ids"]
    else:
        return result

In [ ]:
def handle_single_conversation(conversation):
    input_ids = self.get_prefix_tokens() if add_special_tokens else []
    input_message = "[gMASK]<sop>" if add_special_tokens else ""
    for item in conversation:
        if item.get("tools"):
            tools = item["tools"]
            content = "你是一个名为 GLM-4 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。"
            for tool in tools:
                if tool["type"] == "function":
                    function = tool["function"]
                    content += f"\n\n## {function['name']}\n\n{json.dumps(function, ensure_ascii=False, indent=4)}"
                    content += "\n在调用上述函数时，请使用 Json 格式表示调用的参数。"
                elif tool["type"] == "python":
                    content += "\n\n## python\n\n当你向 `python` 发送包含 Python 代码的消息时，该代码将会在一个有状态的 Jupyter notebook 环境中执行。\n`python` 返回代码执行的输出，或在执行 60 秒后返回超时。\n`/mnt/data` 将会持久化存储你的文件。在此会话中，`python` 无法访问互联网。不要使用 `python` 进行任何网络请求或者在线 API 调用，这些在线内容的访问将不会成功。"
                elif tool["type"] == "simple_browser":
                    content += "\n\n## simple_browser\n\n你可以使用 `simple_browser` 工具。该工具支持以下函数：\n`search(query: str, recency_days: int)`：使用搜索引擎进行查询并显示结果，可以使用 `recency_days` 参数控制搜索内容的时效性。\n`mclick(ids: list[int])`：获取一系列指定 id 的页面内容。每次调用时，须选择3-10个页面。选择多个角度的页面，同时尽可能选择可信任的信息来源。考虑到部分页面是无法加载的，你也可以多打开一些可能有用的页面而不用担心内容过多。\n`open_url(url: str)`：打开指定的 URL。\n\n使用 `【{引用 id}†{引用文本}】` 来引用内容。\n\n操作步骤：1. 使用 `search` 来获得信息列表; 2. 使用 `mclick` 来获取指定 ID 页面的内容; 3. 根据获得的内容进行回复。在回复中应当引用信息来源。\n 如果用户提供了 URL，也可以用 `open_url` 直接打开页面。\n如果初次搜索结果没有找到合适的信息，也可以再次使用 `search` 进行搜索。"
                elif tool["type"] == "cogview":
                    content += "\n\n## cogview\n\n如果用户的请求中包含了对图像的描述，你可以使用 `cogview` 来生成图像并展示给用户。你需要向 `cogview` 发送图像描述，规则：\n- 发送给 `cogview` 的消息必须使用英语。用户的中文描述必须完全翻译为英语。\n- 应当尽可能详细地描述图像生成的需求，需求描述约 100 英文单词。\n- 保持用户原始描述的意图。不要虚构内容或者没见过的人物。\n- 如无特殊说明，所在地为中国，持有中国立场并遵循中国社会主义价值观。"
                else:
                    raise NotImplementedError(f"Unknown tool type {tool['type']}")
            input = self.build_single_message("system", "", content, tokenize=tokenize)
            if tokenize:
                input_ids.extend(input)
            else:
                input_message += input
        if item["content"]:
            input = self.build_single_message(
                item["role"],
                item.get("metadata", ""),
                item["content"],
                tokenize=tokenize
            )
            if tokenize:
                input_ids.extend(input)
            else:
                input_message += input
    if add_generation_prompt:
        if tokenize:
            input_ids.extend([self.convert_tokens_to_ids("<|assistant|>")])
        else:
            input_message += "<|assistant|>"

    return input_ids if tokenize else input_message

In [ ]:
convert_tokens_to_ids("<|assistant|>")

In [58]:
def handle_single_conversation(conversation):
    input_ids = self.get_prefix_tokens() if add_special_tokens else []
    input_message = "[gMASK]<sop>" if add_special_tokens else ""
    for item in conversation:
        if item.get("tools"):
            tools = item["tools"]
            content = "你是一个名为 GLM-4 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。"
            for tool in tools:
                if tool["type"] == "function":
                    function = tool["function"]
                    content += f"\n\n## {function['name']}\n\n{json.dumps(function, ensure_ascii=False, indent=4)}"
                    content += "\n在调用上述函数时，请使用 Json 格式表示调用的参数。"
                elif tool["type"] == "python":
                    content += "\n\n## python\n\n当你向 `python` 发送包含 Python 代码的消息时，该代码将会在一个有状态的 Jupyter notebook 环境中执行。\n`python` 返回代码执行的输出，或在执行 60 秒后返回超时。\n`/mnt/data` 将会持久化存储你的文件。在此会话中，`python` 无法访问互联网。不要使用 `python` 进行任何网络请求或者在线 API 调用，这些在线内容的访问将不会成功。"
                elif tool["type"] == "simple_browser":
                    content += "\n\n## simple_browser\n\n你可以使用 `simple_browser` 工具。该工具支持以下函数：\n`search(query: str, recency_days: int)`：使用搜索引擎进行查询并显示结果，可以使用 `recency_days` 参数控制搜索内容的时效性。\n`mclick(ids: list[int])`：获取一系列指定 id 的页面内容。每次调用时，须选择3-10个页面。选择多个角度的页面，同时尽可能选择可信任的信息来源。考虑到部分页面是无法加载的，你也可以多打开一些可能有用的页面而不用担心内容过多。\n`open_url(url: str)`：打开指定的 URL。\n\n使用 `【{引用 id}†{引用文本}】` 来引用内容。\n\n操作步骤：1. 使用 `search` 来获得信息列表; 2. 使用 `mclick` 来获取指定 ID 页面的内容; 3. 根据获得的内容进行回复。在回复中应当引用信息来源。\n 如果用户提供了 URL，也可以用 `open_url` 直接打开页面。\n如果初次搜索结果没有找到合适的信息，也可以再次使用 `search` 进行搜索。"
                elif tool["type"] == "cogview":
                    content += "\n\n## cogview\n\n如果用户的请求中包含了对图像的描述，你可以使用 `cogview` 来生成图像并展示给用户。你需要向 `cogview` 发送图像描述，规则：\n- 发送给 `cogview` 的消息必须使用英语。用户的中文描述必须完全翻译为英语。\n- 应当尽可能详细地描述图像生成的需求，需求描述约 100 英文单词。\n- 保持用户原始描述的意图。不要虚构内容或者没见过的人物。\n- 如无特殊说明，所在地为中国，持有中国立场并遵循中国社会主义价值观。"
                else:
                    raise NotImplementedError(f"Unknown tool type {tool['type']}")
            input = self.build_single_message("system", "", content, tokenize=tokenize)
            if tokenize:
                input_ids.extend(input)
            else:
                input_message += input
        if item["content"]:
            input = self.build_single_message(
                item["role"],
                item.get("metadata", ""),
                item["content"],
                tokenize=tokenize
            )
            if tokenize:
                input_ids.extend(input)
            else:
                input_message += input
    if add_generation_prompt:
        if tokenize:
            input_ids.extend([self.convert_tokens_to_ids("<|assistant|>")])
        else:
            input_message += "<|assistant|>"
        # if tokenize:
        #     input_ids.extend([self.convert_tokens_to_ids("[gMASK]")])  # 使用特殊标记代替空字符串
        # else:
        #     input_message += "[gMASK]"

    return input_ids if tokenize else input_message

In [60]:
def apply_chat_template(
        self,
        conversation: Union[List[Dict[str, str]], List[List[Dict[str, str]]], "Conversation"],
        add_generation_prompt: bool = False,
        tokenize: bool = True,
        padding: bool = False,
        truncation: bool = False,
        max_length: Optional[int] = None,
        return_tensors: Optional[Union[str, TensorType]] = None,
        return_dict: bool = False,
        tokenizer_kwargs: Optional[Dict[str, Any]] = None,
        add_special_tokens: bool = True,
        **kwargs,
) -> Union[str, List[int], List[str], List[List[int]], BatchEncoding]:

    if return_dict and not tokenize:
        raise ValueError(
            "`return_dict=True` is incompatible with `tokenize=False`, because there is no dict "
            "of tokenizer outputs to return."
        )

    def handle_single_conversation(conversation):
        input_ids = self.get_prefix_tokens() if add_special_tokens else []
        input_message = "[gMASK]<sop>" if add_special_tokens else ""
        for item in conversation:
            if item.get("tools"):
                tools = item["tools"]
                content = "你是一个名为 GLM-4 的人工智能助手。你是基于智谱AI训练的语言模型 GLM-4 模型开发的，你的任务是针对用户的问题和要求提供适当的答复和支持。"
                for tool in tools:
                    if tool["type"] == "function":
                        function = tool["function"]
                        content += f"\n\n## {function['name']}\n\n{json.dumps(function, ensure_ascii=False, indent=4)}"
                        content += "\n在调用上述函数时，请使用 Json 格式表示调用的参数。"
                    elif tool["type"] == "python":
                        content += "\n\n## python\n\n当你向 `python` 发送包含 Python 代码的消息时，该代码将会在一个有状态的 Jupyter notebook 环境中执行。\n`python` 返回代码执行的输出，或在执行 60 秒后返回超时。\n`/mnt/data` 将会持久化存储你的文件。在此会话中，`python` 无法访问互联网。不要使用 `python` 进行任何网络请求或者在线 API 调用，这些在线内容的访问将不会成功。"
                    elif tool["type"] == "simple_browser":
                        content += "\n\n## simple_browser\n\n你可以使用 `simple_browser` 工具。该工具支持以下函数：\n`search(query: str, recency_days: int)`：使用搜索引擎进行查询并显示结果，可以使用 `recency_days` 参数控制搜索内容的时效性。\n`mclick(ids: list[int])`：获取一系列指定 id 的页面内容。每次调用时，须选择3-10个页面。选择多个角度的页面，同时尽可能选择可信任的信息来源。考虑到部分页面是无法加载的，你也可以多打开一些可能有用的页面而不用担心内容过多。\n`open_url(url: str)`：打开指定的 URL。\n\n使用 `【{引用 id}†{引用文本}】` 来引用内容。\n\n操作步骤：1. 使用 `search` 来获得信息列表; 2. 使用 `mclick` 来获取指定 ID 页面的内容; 3. 根据获得的内容进行回复。在回复中应当引用信息来源。\n 如果用户提供了 URL，也可以用 `open_url` 直接打开页面。\n如果初次搜索结果没有找到合适的信息，也可以再次使用 `search` 进行搜索。"
                    elif tool["type"] == "cogview":
                        content += "\n\n## cogview\n\n如果用户的请求中包含了对图像的描述，你可以使用 `cogview` 来生成图像并展示给用户。你需要向 `cogview` 发送图像描述，规则：\n- 发送给 `cogview` 的消息必须使用英语。用户的中文描述必须完全翻译为英语。\n- 应当尽可能详细地描述图像生成的需求，需求描述约 100 英文单词。\n- 保持用户原始描述的意图。不要虚构内容或者没见过的人物。\n- 如无特殊说明，所在地为中国，持有中国立场并遵循中国社会主义价值观。"
                    else:
                        raise NotImplementedError(f"Unknown tool type {tool['type']}")
                input = self.build_single_message("system", "", content, tokenize=tokenize)
                if tokenize:
                    input_ids.extend(input)
                else:
                    input_message += input
            if item["content"]:
                input = self.build_single_message(
                    item["role"],
                    item.get("metadata", ""),
                    item["content"],
                    tokenize=tokenize
                )
                if tokenize:
                    input_ids.extend(input)
                else:
                    input_message += input
        if add_generation_prompt:
            if tokenize:
                input_ids.extend([self.convert_tokens_to_ids("<|assistant|>")])
            else:
                input_message += "<|assistant|>"
            # if tokenize:
            #     input_ids.extend([self.convert_tokens_to_ids("[gMASK]")])  # 使用特殊标记代替空字符串
            # else:
            #     input_message += "[gMASK]"

        return input_ids if tokenize else input_message

    # 处理不同会话格式的主逻辑
    if isinstance(conversation, list) and all(isinstance(i, dict) for i in conversation):
        result = handle_single_conversation(conversation)
    elif isinstance(conversation, list) and all(isinstance(i, list) for i in conversation):
        result = [handle_single_conversation(c) for c in conversation]
    elif hasattr(conversation, "messages"):
        result = handle_single_conversation(conversation.messages)
    else:
        raise ValueError("Invalid conversation format")

    if tokenize:
        output = self.batch_encode_plus(
            [result] if isinstance(result[0], int) else result,
            padding=padding,
            truncation=truncation,
            max_length=max_length,
            return_tensors=return_tensors,
            is_split_into_words=True,
            add_special_tokens=False
        )
        if return_dict:
            return output
        else:
            return output["input_ids"]
    else:
        return result


In [24]:
%pip install regex

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [26]:
pat_str = "(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+"
regex.compile(pat_str)

regex.Regex("(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\\r\\n\\p{L}\\p{N}]?\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]+[\\r\\n]*|\\s*[\\r\\n]+|\\s+(?!\\S)|\\s+", flags=regex.V0)

In [25]:
import regex

pat_str = r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"
pattern = regex.compile(pat_str)